### **5-HT1A-LiCAs: An Interpretable Machine Learning-based Tool for the Identification of Serotonin 5-HT1A Receptor Agonists and Antagonists**

Sumbul Asif , Sara Sarfaraz *, Saba Zafar , Iqra Riaz , Saeed Abo Saeed , Mohamed E. Hasan , Katarzyna Wińska  and Antoni Szumny *

### **Calulation of Morgan Fingerprints uding RDKit:**

Dataset file required: Combined Dataset-Primary.xlsx

In [ ]:
!pip install rdkit
from rdkit import Chem
!pip install openpyxl

In [ ]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem

# Read Excel file
df = pd.read_excel('/content/Combined Dataset-Primary.xlsx')

# Initialize Morgan fingerprint generator
fpgen = AllChem.GetMorganGenerator(radius=2)

print("Generating Morgan fingerprints...")

results = []
failed_drugs = []

for idx, row in df.iterrows():
    smiles = row['SMILE CODE']
    mol = Chem.MolFromSmiles(str(smiles))

    if mol is not None:
        try:
            fp = fpgen.GetFingerprint(mol)
            fp_list = list(fp)
            # Add all original row data plus fingerprints
            new_row = row.to_dict()
            for i, bit in enumerate(fp_list):
                new_row[f'fp_bit_{i}'] = bit
            results.append(new_row)
        except Exception as e:
            # If fingerprint generation fails after molecule creation
            failed_drugs.append({
                'index': idx,
                'smiles': smiles,
                'error': str(e)
            })
    else:
        # If molecule creation fails
        failed_drugs.append({
            'index': idx,
            'smiles': smiles,
            'error': 'Invalid SMILES - cannot parse molecule'
        })

# Create new DataFrame with successful fingerprints
if results:
    result_df = pd.DataFrame(results)
    result_df.to_excel('drugs_with_morgan_fingerprints.xlsx', index=False)
    print(f"✓ Generated fingerprints for {len(result_df)} compounds")
    print("✓ Saved to: drugs_with_morgan_fingerprints.xlsx")
else:
    print("✗ No valid fingerprints generated")

# Print failed drugs details
if failed_drugs:
    print(f"\n✗ Fingerprint generation failed for {len(failed_drugs)} compounds:")
    print("=" * 60)
    for failed in failed_drugs:
        print(f"Row {failed['index'] + 2}: SMILES = '{failed['smiles']}' - {failed['error']}")
else:
    print("✓ All drugs successfully processed!")

print(f"\nSummary:")
print(f"Total compounds processed: {len(df)}")
print(f"Successful: {len(results)}")
print(f"Failed: {len(failed_drugs)}")

## **Calculation of 2D Descriptors using RDKit:**

Dataset required: Combined Dataset-Primary.xlsx

In [ ]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski, Crippen, MolSurf, rdMolDescriptors

# Read Excel file
df = pd.read_excel('/content/Combined Dataset-Primary.xlsx')

print("Calculating ALL 2D molecular descriptors...")

successful_data = []
failed_list = []

for idx, row in df.iterrows():
    smiles = row['SMILE CODE']
    mol = Chem.MolFromSmiles(str(smiles))

    if mol is not None:
        try:
            desc_dict = {}

            # Calculate ALL descriptors from RDKit's descriptor list
            for desc_name, desc_func in Descriptors.descList:
                try:
                    desc_dict[desc_name] = desc_func(mol)
                except:
                    desc_dict[desc_name] = None

            # Store original row data and descriptors
            row_data = row.to_dict()
            row_data.update(desc_dict)
            successful_data.append(row_data)

        except Exception as e:
            failed_list.append((idx, smiles, str(e)))
    else:
        failed_list.append((idx, smiles, 'Invalid SMILES'))

if successful_data:
    # Create result DataFrame
    result_df = pd.DataFrame(successful_data)

    # Get the position of SMILE CODE column
    original_cols = list(df.columns)
    smile_idx = original_cols.index('SMILE CODE')

    # Get descriptor columns (all columns not in original DataFrame)
    desc_cols = [col for col in result_df.columns if col not in original_cols]

    # Create new column order: original columns up to SMILE CODE, then descriptors, then remaining original columns
    new_column_order = (original_cols[:smile_idx+1] +
                       desc_cols +
                       original_cols[smile_idx+1:])

    # Reorder columns
    result_df = result_df[new_column_order]

    # Save to Excel
    result_df.to_excel('drugs_with_all_2d_descriptors.xlsx', index=False)

    print(f"✓ Generated {len(desc_cols)} 2D descriptors for {len(result_df)} compounds")
    print("✓ Descriptors placed right after 'SMILE CODE' column")
    print("✓ Saved to: drugs_with_all_2d_descriptors.xlsx")

    # Show descriptor count and names
    print(f"\nTotal descriptors calculated: {len(desc_cols)}")
    print(f"First 20 descriptors: {desc_cols[:20]}")

# Print failed compounds
if failed_list:
    print(f"\n✗ Failed for {len(failed_list)} compounds:")
    for idx, smiles, error in failed_list:
        print(f"Row {idx+2}: '{smiles}' - {error}")

print(f"\nSummary: {len(successful_data)} successful, {len(failed_list)} failed")

### **Calculation of 3D Descriptors using RDKit:**

Dataset required: Combined Dataset-primary.xlsx

In [ ]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem, rdMolDescriptors, Descriptors
import numpy as np

# Read Excel file
df = pd.read_excel('/content/Combined Dataset-Primary.xlsx')

print("Calculating 3D molecular descriptors...")

successful_data = []
failed_list = []

for idx, row in df.iterrows():
    smiles = row['SMILE CODE']
    mol = Chem.MolFromSmiles(str(smiles))

    if mol is not None:
        try:
            desc_dict = {}

            # Generate 3D conformation
            mol_3d = Chem.AddHs(mol)  # Add hydrogens for 3D
            AllChem.EmbedMolecule(mol_3d, randomSeed=42)  # Generate 3D coordinates
            AllChem.MMFFOptimizeMolecule(mol_3d)  # Energy minimization

            # ===== 3D DESCRIPTORS =====

            # PMI (Principal Moment of Inertia) descriptors
            pmi1 = rdMolDescriptors.CalcPMI1(mol_3d)
            pmi2 = rdMolDescriptors.CalcPMI2(mol_3d)
            pmi3 = rdMolDescriptors.CalcPMI3(mol_3d)
            desc_dict['PMI1'] = pmi1
            desc_dict['PMI2'] = pmi2
            desc_dict['PMI3'] = pmi3
            desc_dict['NPR1'] = pmi1 / pmi3 if pmi3 != 0 else 0
            desc_dict['NPR2'] = pmi2 / pmi3 if pmi3 != 0 else 0

            # Radius of gyration
            desc_dict['RadiusOfGyration'] = rdMolDescriptors.CalcRadiusOfGyration(mol_3d)

            # Inertial shape factor
            desc_dict['InertialShapeFactor'] = rdMolDescriptors.CalcInertialShapeFactor(mol_3d)

            # Eccentricity
            desc_dict['Eccentricity'] = rdMolDescriptors.CalcEccentricity(mol_3d)

            # Asphericity
            desc_dict['Asphericity'] = rdMolDescriptors.CalcAsphericity(mol_3d)

            # Spherocity index
            desc_dict['SpherocityIndex'] = rdMolDescriptors.CalcSpherocityIndex(mol_3d)

            # Plane of best fit
            desc_dict['PBF'] = rdMolDescriptors.CalcPBF(mol_3d)

            # Remove unavailable descriptors
            # desc_dict['Wiener3D'] = rdMolDescriptors.Calc3DWiener(mol_3d)  # Not available
            # desc_dict['Balaban3D'] = rdMolDescriptors.Calc3DBalaban(mol_3d)  # Not available

            # Hydrophobic surface area descriptors
            desc_dict['FractionCSP3_3D'] = rdMolDescriptors.CalcFractionCSP3(mol_3d)

            # Molecular surface area and volume
            desc_dict['LabuteASA'] = rdMolDescriptors.CalcLabuteASA(mol_3d)
            desc_dict['TPSA_3D'] = rdMolDescriptors.CalcTPSA(mol_3d)

            # Get conformer and calculate geometric descriptors
            if mol_3d.GetNumConformers() > 0:
                conf = mol_3d.GetConformer()

                # Calculate geometric center
                coords = [list(conf.GetAtomPosition(i)) for i in range(mol_3d.GetNumAtoms())]
                if coords:
                    center = np.mean(coords, axis=0)
                    desc_dict['GeometricCenter_X'] = center[0]
                    desc_dict['GeometricCenter_Y'] = center[1]
                    desc_dict['GeometricCenter_Z'] = center[2]

                    # Calculate molecular dimensions
                    coords_array = np.array(coords)
                    min_coords = np.min(coords_array, axis=0)
                    max_coords = np.max(coords_array, axis=0)
                    dimensions = max_coords - min_coords
                    desc_dict['MolecularLength'] = dimensions[0]
                    desc_dict['MolecularWidth'] = dimensions[1]
                    desc_dict['MolecularHeight'] = dimensions[2]
                    desc_dict['MolecularVolume_Box'] = dimensions[0] * dimensions[1] * dimensions[2]

            # Store original row data and descriptors
            row_data = row.to_dict()
            row_data.update(desc_dict)
            successful_data.append(row_data)

        except Exception as e:
            failed_list.append((idx, smiles, str(e)))
    else:
        failed_list.append((idx, smiles, 'Invalid SMILES'))

if successful_data:
    # Create result DataFrame
    result_df = pd.DataFrame(successful_data)

    # Get the position of SMILE CODE column
    original_cols = list(df.columns)
    smile_idx = original_cols.index('SMILE CODE')

    # Get descriptor columns (all columns not in original DataFrame)
    desc_cols = [col for col in result_df.columns if col not in original_cols]

    # Create new column order: original columns up to SMILE CODE, then descriptors, then remaining original columns
    new_column_order = (original_cols[:smile_idx+1] +
                       desc_cols +
                       original_cols[smile_idx+1:])

    # Reorder columns
    result_df = result_df[new_column_order]

    # Save to Excel
    result_df.to_excel('drugs_with_3d_descriptors.xlsx', index=False)

    print(f"✓ Generated {len(desc_cols)} 3D descriptors for {len(result_df)} compounds")
    print("✓ Descriptors placed right after 'SMILE CODE' column")
    print("✓ Saved to: drugs_with_3d_descriptors.xlsx")

    # Show descriptor names
    print(f"\n3D descriptors calculated: {desc_cols}")

# Print failed compounds
if failed_list:
    print(f"\n✗ Failed for {len(failed_list)} compounds:")
    for idx, smiles, error in failed_list:
        print(f"Row {idx+2}: '{smiles}' - {error}")

print(f"\nSummary: {len(successful_data)} successful, {len(failed_list)} failed")

### **Some Settings to install Additional Fonts for Images:**

In [ ]:
# ============================================================================
# GOOGLE COLAB FONT SETUP FOR PUBLICATION-QUALITY PLOTS
# ============================================================================
print("🔧 Setting up fonts for publication-quality plots...")

# 1. Install the Liberation Sans font package (open-source, Arial-compatible)
!sudo apt-get update -qq
!sudo apt-get install -y fonts-liberation
print("✅ Font package installed.")

# 2. Clear matplotlib's font cache to force it to recognize new fonts
import matplotlib
!rm -rf ~/.cache/matplotlib

# 3. Configure matplotlib to use Liberation Sans as the primary sans-serif font
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Liberation Sans', 'DejaVu Sans', 'Verdana']
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['axes.titlesize'] = 14

# 4. Optional: Verify the font is available
import matplotlib.font_manager as fm
available_fonts = [f.name for f in fm.fontManager.ttflist]
liberation_available = any('Liberation' in f for f in available_fonts)
print(f"✅ Liberation Sans available: {liberation_available}")

print("✅ Font setup complete. Your plots will now use Liberation Sans.\n")

# ============================================================================
# YOUR ORIGINAL PLOTTING CODE STARTS BELOW
# ============================================================================
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


### **Dataset Distribution Plot for Complete Dataset:**

Dataset required:Dataset for Machine Learning.xlsx

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Read your data
df = pd.read_excel('/content/Dataset for Machine Learning.xlsx')

print("Dataset Overview:")
print(f"Total compounds: {len(df)}")

# Count instances in each class
class_counts = df['Ligand Type'].value_counts().sort_index()
class_names = {0: 'Antagonist', 1: 'Agonist'}
class_counts_named = class_counts.rename(index=class_names)

print(f"Agonists (1): {class_counts[1]}")
print(f"Antagonists (0): {class_counts[0]}")
print(f"Agonist/Antagonist ratio: {class_counts[1]/class_counts[0]:.2f}:1")

# Create a single figure with larger size
plt.figure(figsize=(12, 8))

# Plot the single bar plot
bars = plt.bar(class_counts_named.index, class_counts_named.values,
               color=['#ff6b6b', '#4ecdc4'], alpha=0.8, edgecolor='black', linewidth=2)

plt.title('Distribution of Agonists vs Antagonists', fontsize=20, fontweight='bold', pad=20)
plt.ylabel('Number of Compounds', fontsize=18, fontweight='bold')
plt.xlabel('Ligand Type', fontsize=18, fontweight='bold')

# Increase tick label sizes
plt.xticks(fontsize=16, fontweight='bold')
plt.yticks(fontsize=16)

# Add value labels on bars with larger font
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 0.5,
             f'{int(height)}', ha='center', va='bottom',
             fontweight='bold', fontsize=16)

# Add grid for better readability
plt.grid(axis='y', alpha=0.3, linestyle='--')

# Adjust layout and show plot
plt.tight_layout()
plt.show()

# Print detailed statistics
print("\n" + "="*50)
print("DETAILED CLASS DISTRIBUTION STATISTICS")
print("="*50)
print(f"Total compounds: {len(df)}")
print(f"Antagonists (0): {class_counts[0]} ({class_counts[0]/len(df)*100:.1f}%)")
print(f"Agonists (1): {class_counts[1]} ({class_counts[1]/len(df)*100:.1f}%)")
print(f"Class ratio (Agonist:Antagonist): {class_counts[1]/class_counts[0]:.2f}:1")

# Check for class imbalance
if class_counts[0] / class_counts[1] > 2 or class_counts[1] / class_counts[0] > 2:
    print("\n⚠️  WARNING: Significant class imbalance detected!")
    print("   Consider using class weights in your ML model.")
else:
    print("\n✓ Balanced dataset - good for machine learning!")

### **Test-Train Split:**

Dataset required:Dataset for Machine Learning.xlsx

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

# Read your data
df = pd.read_excel('/content/Dataset for Machine Learning.xlsx')

print("Original Dataset Overview:")
print(f"Total compounds: {len(df)}")
print(f"Columns in dataset: {list(df.columns)}")

# Check class distribution
class_counts = df['Ligand Type'].value_counts().sort_index()
print(f"Antagonists (0): {class_counts[0]} ({class_counts[0]/len(df)*100:.1f}%)")
print(f"Agonists (1): {class_counts[1]} ({class_counts[1]/len(df)*100:.1f}%)")

# Prepare features and target - using exact column names from your dataset
X = df.drop(['COMPOUND ID', 'SMILE CODE', 'Ligand Type'], axis=1)
y = df['Ligand Type']

# Split the data (80% train, 20% test) with stratification
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y  # This maintains the same class distribution in both sets
)

print(f"\nSplit Results:")
print(f"Training set: {X_train.shape}")
print(f"Testing set: {X_test.shape}")

# Create complete DataFrames for training and testing
train_df = pd.DataFrame({
    'COMPOUND ID': df.iloc[X_train.index]['COMPOUND ID'].values,
    'SMILE CODE': df.iloc[X_train.index]['SMILE CODE'].values
})
train_df = pd.concat([train_df, X_train.reset_index(drop=True), y_train.reset_index(drop=True)], axis=1)

test_df = pd.DataFrame({
    'COMPOUND ID': df.iloc[X_test.index]['COMPOUND ID'].values,
    'SMILE CODE': df.iloc[X_test.index]['SMILE CODE'].values
})
test_df = pd.concat([test_df, X_test.reset_index(drop=True), y_test.reset_index(drop=True)], axis=1)

# Verify the splits
print("\nTraining Set Class Distribution:")
train_counts = train_df['Ligand Type'].value_counts().sort_index()
print(f"Antagonists (0): {train_counts[0]} ({train_counts[0]/len(train_df)*100:.1f}%)")
print(f"Agonists (1): {train_counts[1]} ({train_counts[1]/len(train_df)*100:.1f}%)")

print("\nTesting Set Class Distribution:")
test_counts = test_df['Ligand Type'].value_counts().sort_index()
print(f"Antagonists (0): {test_counts[0]} ({test_counts[0]/len(test_df)*100:.1f}%)")
print(f"Agonists (1): {test_counts[1]} ({test_counts[1]/len(test_df)*100:.1f}%)")

# Save to Excel files
train_df.to_excel('training_set.xlsx', index=False)
test_df.to_excel('testing_set.xlsx', index=False)

print(f"\n✓ Files saved successfully!")
print(f"Training set: training_set.xlsx ({len(train_df)} samples)")
print(f"Testing set: testing_set.xlsx ({len(test_df)} samples)")

# Also save the feature matrices and targets separately for ML
np.save('X_train.npy', X_train.values)
np.save('X_test.npy', X_test.values)
np.save('y_train.npy', y_train.values)
np.save('y_test.npy', y_test.values)

print(f"✓ Numpy arrays saved for ML modeling:")
print(f"  X_train.npy, X_test.npy, y_train.npy, y_test.npy")

### **Dataset Distribution in Training and Testing Data:**

Dataset required:Dataset for Machine Learning.xlsx

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Read data and split
df = pd.read_excel('/content/Dataset for Machine Learning.xlsx')
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['Ligand Type'])

# Calculate counts for the plot
train_counts = train_df['Ligand Type'].value_counts().sort_index()
test_counts = test_df['Ligand Type'].value_counts().sort_index()

# Create the comparison plot
plt.figure(figsize=(10, 6))
bar_width = 0.35
x_pos = np.arange(2)

plt.bar(x_pos - bar_width/2, train_counts.values, bar_width,
        label='Training Set', color='#3498db', alpha=0.8, edgecolor='black')
plt.bar(x_pos + bar_width/2, test_counts.values, bar_width,
        label='Testing Set', color='#f39c12', alpha=0.8, edgecolor='black')

plt.title('CLASS DISTRIBUTION COMPARISON: Training vs Testing Sets',
          fontsize=14, fontweight='bold')
plt.xlabel('Ligand Type', fontsize=12)
plt.ylabel('Number of Compounds', fontsize=12)
plt.xticks(x_pos, ['Antagonist (0)', 'Agonist (1)'])
plt.legend()
plt.grid(axis='y', alpha=0.3)

# Add value labels on bars
for i, v in enumerate(train_counts.values):
    plt.text(i - bar_width/2, v + 0.5, str(v), ha='center', va='bottom', fontweight='bold')
for i, v in enumerate(test_counts.values):
    plt.text(i + bar_width/2, v + 0.5, str(v), ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

### **Features Selectin using ReliefF**

Dataset required:Dataset for Machine Learning.xlsx

In [ ]:
!pip install skrebate

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from skrebate import ReliefF
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

# Set publication-quality parameters
plt.rcParams['font.size'] = 12
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['axes.titlesize'] = 16
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
plt.rcParams['legend.fontsize'] = 12
plt.rcParams['figure.titlesize'] = 18

# Professional color scheme
colors = ['#2E86AB', '#A23B72', '#F18F01', '#C73E1D', '#3E92CC']

print("🎯 FEATURE SELECTION PIPELINE")
print("="*70)

# Load dataset
data_df = pd.read_excel('/content/Dataset for Machine Learning.xlsx')

print("📊 Dataset Information:")
print(f"Complete dataset: {data_df.shape}")

# STEP 1: Identify feature types based on your exact column structure
print("\n" + "="*70)
print("🔍 IDENTIFYING FEATURE TYPES")
print("="*70)

# Get all columns
all_columns = data_df.columns.tolist()

# Identify feature sets based on your exact column ranges
morgan_columns = [col for col in all_columns if col.startswith('fp_bit_')]

# Find the exact range for 2D descriptors
try:
    max_abs_index = all_columns.index('MaxAbsEStateIndex')
    fr_urea_index = all_columns.index('fr_urea')
    td_columns = all_columns[max_abs_index:fr_urea_index + 1]
except ValueError as e:
    print(f"Error finding 2D descriptor boundaries: {e}")
    td_columns = []

# Find the exact range for 3D descriptors
try:
    pmi1_index = all_columns.index('PMI1')
    mv_box_index = all_columns.index('MolecularVolume_Box')
    threed_columns = all_columns[pmi1_index:mv_box_index + 1]
except ValueError as e:
    print(f"Error finding 3D descriptor boundaries: {e}")
    threed_columns = []

print(f"✅ FEATURE SETS IDENTIFIED:")
print(f"   Morgan Fingerprints: {len(morgan_columns)} features (fp_bit_0 to fp_bit_2047)")
print(f"   2D Descriptors: {len(td_columns)} features (MaxAbsEStateIndex to fr_urea)")
print(f"   3D Descriptors: {len(threed_columns)} features (PMI1 to MolecularVolume_Box)")

# STEP 2: Train-Test Split
print("\n" + "="*70)
print("📊 TRAIN-TEST SPLIT")
print("="*70)

# Prepare features and target
metadata_columns = ['COMPOUND ID', 'SMILE CODE', 'Ligand Type']
X = data_df.drop(metadata_columns, axis=1)
y = data_df['Ligand Type']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training set: {X_train.shape}")
print(f"Testing set: {X_test.shape}")

# STEP 3: Feature Selection using ReliefF
print("\n" + "="*70)
print("🎯 FEATURE SELECTION USING RELIEFF")
print("="*70)

def perform_relieff_selection(X_train, y_train, feature_columns, n_features_to_select=100, feature_set_name=""):
    """Perform ReliefF feature selection on specified feature set"""
    print(f"\n🔍 Selecting features for {feature_set_name}:")
    print(f"   Total features: {len(feature_columns)}")

    if len(feature_columns) <= n_features_to_select:
        print(f"   ✅ Keeping all {len(feature_columns)} features")
        return feature_columns, pd.DataFrame({
            'Feature': feature_columns,
            'ReliefF_Score': [1.0] * len(feature_columns)
        })

    print(f"   Selecting top {n_features_to_select} features using ReliefF...")

    # Extract the specific features from training set
    X_subset = X_train[feature_columns].values

    # Encode labels for ReliefF
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y_train)

    # Apply ReliefF
    relief = ReliefF(
        n_features_to_select=n_features_to_select,
        n_neighbors=100
    )

    relief.fit(X_subset, y_encoded)

    # Get top feature indices and scores
    top_indices = relief.top_features_[:n_features_to_select]
    top_scores = relief.feature_importances_[top_indices]

    selected_features = [feature_columns[i] for i in top_indices]

    print(f"   ✅ Selected {len(selected_features)} features")

    # Create feature importance dataframe for this set
    importance_df = pd.DataFrame({
        'Feature': selected_features,
        'ReliefF_Score': top_scores
    }).sort_values('ReliefF_Score', ascending=False)

    return selected_features, importance_df

# Perform feature selection for each set
selected_features = {}
feature_importance_dfs = {}

# Morgan Fingerprints: Select top 100 (CHANGED FROM 30 TO 100)
print("🔄 Processing Morgan Fingerprints...")
morgan_selected, morgan_importance = perform_relieff_selection(
    X_train, y_train, morgan_columns, 100, "Morgan Fingerprints"  # Changed to 100
)
selected_features['morgan'] = morgan_selected
feature_importance_dfs['morgan'] = morgan_importance

# 2D Descriptors: Select top 100 (CHANGED FROM 30 TO 100)
print("🔄 Processing 2D Descriptors...")
td_selected, td_importance = perform_relieff_selection(
    X_train, y_train, td_columns, 100, "2D Descriptors"  # Changed to 100
)
selected_features['2d_descriptors'] = td_selected
feature_importance_dfs['2d_descriptors'] = td_importance

# 3D Descriptors: Keep all (as per your specification)
print("🔄 Processing 3D Descriptors...")
print(f"   Total features: {len(threed_columns)}")
print(f"   ✅ Keeping all {len(threed_columns)} features (as specified)")
selected_features['3d_descriptors'] = threed_columns

# Create importance dataframe for 3D (all features with dummy scores)
threed_importance = pd.DataFrame({
    'Feature': threed_columns,
    'ReliefF_Score': [1.0] * len(threed_columns)  # Placeholder score
})
feature_importance_dfs['3d_descriptors'] = threed_importance

# Combine all selected features
all_selected_features = (
    selected_features['morgan'] +
    selected_features['2d_descriptors'] +
    selected_features['3d_descriptors']
)

print(f"\n🎯 FINAL FEATURE SELECTION SUMMARY:")
print(f"   Total selected features: {len(all_selected_features)}")
print(f"   - Morgan Fingerprints: {len(selected_features['morgan'])}")
print(f"   - 2D Descriptors: {len(selected_features['2d_descriptors'])}")
print(f"   - 3D Descriptors: {len(selected_features['3d_descriptors'])}")

# STEP 4: Create new datasets with selected features
print("\n" + "="*70)
print("📊 CREATING FEATURE-SELECTED DATASETS")
print("="*70)

X_train_selected = X_train[all_selected_features]
X_test_selected = X_test[all_selected_features]

print(f"Training set after feature selection: {X_train_selected.shape}")
print(f"Testing set after feature selection: {X_test_selected.shape}")

# STEP 5: Save all results
print("\n" + "="*70)
print("💾 SAVING FEATURE SELECTION RESULTS")
print("="*70)

# Save selected features list
selected_features_df = pd.DataFrame({
    'Feature_Type': (['Morgan'] * len(selected_features['morgan']) +
                    ['2D_Descriptors'] * len(selected_features['2d_descriptors']) +
                    ['3D_Descriptors'] * len(selected_features['3d_descriptors'])),
    'Feature_Name': all_selected_features
})
selected_features_df.to_excel('selected_features_complete.xlsx', index=False)
print("✅ Saved: selected_features_complete.xlsx")

# Save feature importance scores for each set
with pd.ExcelWriter('feature_importance_scores.xlsx') as writer:
    feature_importance_dfs['morgan'].to_excel(writer, sheet_name='Morgan_Fingerprints', index=False)
    feature_importance_dfs['2d_descriptors'].to_excel(writer, sheet_name='2D_Descriptors', index=False)
    feature_importance_dfs['3d_descriptors'].to_excel(writer, sheet_name='3D_Descriptors', index=False)
print("✅ Saved: feature_importance_scores.xlsx")

# Save the train-test splits with selected features
train_selected_df = pd.concat([
    data_df.loc[X_train.index, metadata_columns].reset_index(drop=True),
    X_train_selected.reset_index(drop=True)
], axis=1)

test_selected_df = pd.concat([
    data_df.loc[X_test.index, metadata_columns].reset_index(drop=True),
    X_test_selected.reset_index(drop=True)
], axis=1)

train_selected_df.to_excel('training_set_selected_features.xlsx', index=False)
test_selected_df.to_excel('testing_set_selected_features.xlsx', index=False)
print("✅ Saved: training_set_selected_features.xlsx")
print("✅ Saved: testing_set_selected_features.xlsx")

# Save the feature selection objects for reuse
feature_selection_artifacts = {
    'selected_features': selected_features,
    'all_selected_features': all_selected_features,
    'feature_columns_mapping': {
        'morgan_original': morgan_columns,
        '2d_original': td_columns,
        '3d_original': threed_columns
    }
}
joblib.dump(feature_selection_artifacts, 'feature_selection_artifacts.pkl')
print("✅ Saved: feature_selection_artifacts.pkl")

# STEP 6: Create comprehensive visualization
print("\n" + "="*70)
print("📊 CREATING FEATURE SELECTION VISUALIZATION")
print("="*70)

plt.figure(figsize=(15, 10))

# Plot 1: Feature selection summary
plt.subplot(2, 2, 1)
feature_types = ['Morgan Fingerprints', '2D Descriptors', '3D Descriptors']
original_counts = [len(morgan_columns), len(td_columns), len(threed_columns)]
selected_counts = [len(selected_features['morgan']),
                  len(selected_features['2d_descriptors']),
                  len(selected_features['3d_descriptors'])]

x = np.arange(len(feature_types))
width = 0.35

bars1 = plt.bar(x - width/2, original_counts, width, label='Original',
                color=colors[0], alpha=0.8, edgecolor='black')
bars2 = plt.bar(x + width/2, selected_counts, width, label='Selected',
                color=colors[1], alpha=0.8, edgecolor='black')

plt.ylabel('Number of Features')
plt.title('Feature Selection Summary', fontweight='bold')
plt.xticks(x, feature_types)
plt.legend()
plt.grid(True, alpha=0.3, axis='y')

# Add value labels
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2, height + 5,
                f'{int(height)}', ha='center', va='bottom', fontweight='bold')

# Plot 2: Top features from Morgan fingerprints
plt.subplot(2, 2, 2)
top_morgan = feature_importance_dfs['morgan'].head(15)
plt.barh(range(len(top_morgan)), top_morgan['ReliefF_Score'],
         color=colors[2], alpha=0.8, edgecolor='black')
plt.yticks(range(len(top_morgan)), [f"Bit {f.split('_')[-1]}" for f in top_morgan['Feature']])
plt.xlabel('ReliefF Importance Score')
plt.title('Top 15 Morgan Fingerprints', fontweight='bold')
plt.gca().invert_yaxis()
plt.grid(True, alpha=0.3, axis='x')

# Plot 3: Top features from 2D descriptors
plt.subplot(2, 2, 3)
top_2d = feature_importance_dfs['2d_descriptors'].head(15)
plt.barh(range(len(top_2d)), top_2d['ReliefF_Score'],
         color=colors[3], alpha=0.8, edgecolor='black')
plt.yticks(range(len(top_2d)), top_2d['Feature'])
plt.xlabel('ReliefF Importance Score')
plt.title('Top 15 2D Descriptors', fontweight='bold')
plt.gca().invert_yaxis()
plt.grid(True, alpha=0.3, axis='x')

# Plot 4: Dataset dimensions comparison
plt.subplot(2, 2, 4)
dimensions = ['Original', 'After Selection']
train_sizes = [X_train.shape[1], X_train_selected.shape[1]]
test_sizes = [X_test.shape[1], X_test_selected.shape[1]]

x = np.arange(len(dimensions))
width = 0.35

bars1 = plt.bar(x - width/2, train_sizes, width, label='Training',
                color=colors[0], alpha=0.8, edgecolor='black')
bars2 = plt.bar(x + width/2, test_sizes, width, label='Testing',
                color=colors[1], alpha=0.8, edgecolor='black')

plt.ylabel('Number of Features')
plt.title('Dataset Dimensions Comparison', fontweight='bold')
plt.xticks(x, dimensions)
plt.legend()
plt.grid(True, alpha=0.3, axis='y')

# Add value labels
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2, height + 10,
                f'{int(height)}', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.savefig('feature_selection_comprehensive.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n🎯 FEATURE SELECTION PIPELINE COMPLETED!")
print("="*70)
print(f"📊 RESULTS SUMMARY:")
print(f"   • Original features: {X.shape[1]}")
print(f"   • Selected features: {len(all_selected_features)}")
print(f"   • Reduction: {X.shape[1] - len(all_selected_features)} features removed")
print(f"   • Morgan fingerprints: {len(selected_features['morgan'])} selected from {len(morgan_columns)}")
print(f"   • 2D descriptors: {len(selected_features['2d_descriptors'])} selected from {len(td_columns)}")
print(f"   • 3D descriptors: {len(selected_features['3d_descriptors'])} kept from {len(threed_columns)}")

print(f"\n💾 FILES GENERATED:")
print(f"   1. selected_features_complete.xlsx - Complete list of selected features")
print(f"   2. feature_importance_scores.xlsx - ReliefF scores for each feature set")
print(f"   3. training_set_selected_features.xlsx - Training set with selected features")
print(f"   4. testing_set_selected_features.xlsx - Testing set with selected features")
print(f"   5. feature_selection_artifacts.pkl - Objects for reproducibility")
print(f"   6. feature_selection_comprehensive.png - Visualization of results")

print(f"\n🚀 You can now use these feature-selected datasets for all your model training!")
print(f"📁 Use 'training_set_selected_features.xlsx' and 'testing_set_selected_features.xlsx'")
print(f"   for ANN, Random Forest, KNN, and other models!")

### **Random Forest Classifier:**

Requirements:
training-set-selected-features.xlsx,
testing-set-selected-featres.xlsx

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                           roc_auc_score, roc_curve, matthews_corrcoef, confusion_matrix,
                           classification_report)
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score, StratifiedKFold, cross_val_predict, GridSearchCV
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

# Set publication-quality parameters (Fixed font settings)
plt.rcParams['font.size'] = 12
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['axes.titlesize'] = 16
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
plt.rcParams['legend.fontsize'] = 12
plt.rcParams['figure.titlesize'] = 18

# Use available system fonts to avoid warnings
plt.rcParams['font.family'] = ['DejaVu Sans', 'Arial', 'Helvetica', 'sans-serif']

# Professional Elsevier color scheme
elsevier_colors = ['#2E86AB', '#A23B72', '#F18F01', '#C73E1D', '#1A936F', '#114B5F']

print("🚀 RANDOM FOREST CLASSIFIER WITH HYPERPARAMETER TUNING (10-FOLD CV + GRID SEARCH)")
print("="*80)

# STEP 1: Load the feature-selected datasets
print("📊 Loading feature-selected datasets...")

train_df = pd.read_excel('training_set_selected_features.xlsx')
test_df = pd.read_excel('testing_set_selected_features.xlsx')

print(f"Training set: {train_df.shape}")
print(f"Testing set: {test_df.shape}")

# STEP 2: Prepare features and target
print("\n" + "="*80)
print("🔧 PREPARING DATA")
print("="*80)

# Identify feature columns (exclude metadata)
metadata_columns = ['COMPOUND ID', 'SMILE CODE', 'Ligand Type']
feature_columns = [col for col in train_df.columns if col not in metadata_columns]

X_train = train_df[feature_columns]
y_train = train_df['Ligand Type']

X_test = test_df[feature_columns]
y_test = test_df['Ligand Type']

print(f"Selected features: {len(feature_columns)}")
print(f"Training samples: {X_train.shape[0]}")
print(f"Testing samples: {X_test.shape[0]}")

# Encode labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# FIXED: Properly get class names as strings
class_names = [str(cls) for cls in label_encoder.classes_]
print(f"Class names: {class_names}")

print("✅ Data prepared successfully!")

# STEP 3: Hyperparameter Tuning with GridSearchCV and 10-Fold CV
print("\n" + "="*80)
print("🎯 HYPERPARAMETER TUNING WITH GRIDSEARCHCV (10-FOLD CV)")
print("="*80)

# Define parameter grid for Random Forest
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', None],
    'bootstrap': [True, False]
}

# Initialize Random Forest
rf = RandomForestClassifier(random_state=42)

# 10-fold cross-validation strategy
cv_strategy = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# GridSearchCV with 10-fold CV
print("🔍 Performing Grid Search with 10-fold CV...")
print(f"   Parameter grid: {param_grid}")
print(f"   Total parameter combinations: {len(param_grid['n_estimators']) * len(param_grid['max_depth']) * len(param_grid['min_samples_split']) * len(param_grid['min_samples_leaf']) * len(param_grid['max_features']) * len(param_grid['bootstrap'])}")
print(f"   This may take a while...")

grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    cv=cv_strategy,
    scoring='accuracy',
    n_jobs=-1,
    verbose=1
)

# Perform grid search
grid_search.fit(X_train, y_train_encoded)

print("✅ Grid Search completed!")

# Display best parameters and scores
print(f"\n🎯 BEST PARAMETERS FOUND:")
best_params = grid_search.best_params_
for param, value in best_params.items():
    print(f"   {param}: {value}")
print(f"   Best CV Score: {grid_search.best_score_:.4f}")

# STEP 4: Compare Default vs Tuned Models
print("\n" + "="*80)
print("📊 COMPARISON: DEFAULT vs TUNED PARAMETERS")
print("="*80)

# Train model with default parameters for comparison
rf_default = RandomForestClassifier(random_state=42)
rf_default.fit(X_train, y_train_encoded)
y_pred_default = rf_default.predict(X_test)
accuracy_default = accuracy_score(y_test_encoded, y_pred_default)

# Use best model from grid search
best_rf_model = grid_search.best_estimator_
y_pred_tuned = best_rf_model.predict(X_test)
accuracy_tuned = accuracy_score(y_test_encoded, y_pred_tuned)

improvement = accuracy_tuned - accuracy_default

print(f"   Default Parameters Accuracy: {accuracy_default:.4f}")
print(f"   Tuned Parameters Accuracy:   {accuracy_tuned:.4f}")
print(f"   Improvement:                 {improvement:+.4f}")

if improvement > 0:
    print(f"   ✅ Tuning improved accuracy by {improvement:.2%}")
else:
    print(f"   ⚠️  Tuning did not improve accuracy")

# STEP 5: 10-Fold Cross-Validation with Tuned Model (for proper comparison)
print("\n" + "="*80)
print("🎯 10-FOLD CROSS-VALIDATION WITH TUNED PARAMETERS")
print("="*80)

print("🚀 Performing 10-fold cross-validation with tuned parameters...")

# Get cross-validated predictions for training set using tuned parameters
cv_predictions = cross_val_predict(best_rf_model, X_train, y_train_encoded,
                                 cv=cv_strategy, method='predict')
cv_probabilities = cross_val_predict(best_rf_model, X_train, y_train_encoded,
                                   cv=cv_strategy, method='predict_proba')

# Calculate training metrics as average across 10-fold CV
train_accuracy = accuracy_score(y_train_encoded, cv_predictions)

# Calculate metrics for BOTH CLASSES
train_precision = precision_score(y_train_encoded, cv_predictions, average=None)
train_recall = recall_score(y_train_encoded, cv_predictions, average=None)
train_f1 = f1_score(y_train_encoded, cv_predictions, average=None)
train_roc_auc = roc_auc_score(y_train_encoded, cv_probabilities[:, 1])
train_mcc = matthews_corrcoef(y_train_encoded, cv_predictions)

# Also get individual fold accuracies for reporting
cv_scores = cross_val_score(best_rf_model, X_train, y_train_encoded,
                          cv=cv_strategy, scoring='accuracy', n_jobs=-1)

print("✅ 10-fold cross-validation completed!")

print(f"\n📊 10-FOLD CROSS-VALIDATION RESULTS (TRAINING PERFORMANCE):")
print(f"   Fold scores: {[f'{score:.4f}' for score in cv_scores]}")
print(f"   Mean CV Accuracy: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")
print(f"   Standard Deviation: {cv_scores.std():.4f}")

# STEP 6: Make predictions on TEST set with tuned model
print("\n" + "="*80)
print("📊 MODEL EVALUATION WITH TUNED PARAMETERS")
print("="*80)

# Predictions on TEST set with tuned model
y_test_pred = best_rf_model.predict(X_test)
y_test_proba = best_rf_model.predict_proba(X_test)

# Calculate metrics for TESTING set for BOTH CLASSES
test_accuracy = accuracy_score(y_test_encoded, y_test_pred)
test_precision = precision_score(y_test_encoded, y_test_pred, average=None)
test_recall = recall_score(y_test_encoded, y_test_pred, average=None)
test_f1 = f1_score(y_test_encoded, y_test_pred, average=None)
test_roc_auc = roc_auc_score(y_test_encoded, y_test_proba[:, 1])
test_mcc = matthews_corrcoef(y_test_encoded, y_test_pred)

# Confusion matrices
train_cm = confusion_matrix(y_train_encoded, cv_predictions)
test_cm = confusion_matrix(y_test_encoded, y_test_pred)
tn, fp, fn, tp = test_cm.ravel()

# Additional metrics
specificity = tn / (tn + fp)
npv = tn / (tn + fn)

# Performance gap
accuracy_gap = train_accuracy - test_accuracy

# Compute ROC curves for later saving
fpr_train, tpr_train, _ = roc_curve(y_train_encoded, cv_probabilities[:, 1])
fpr_test, tpr_test, _ = roc_curve(y_test_encoded, y_test_proba[:, 1])

# STEP 7: Display comprehensive results for BOTH CLASSES
print("\n" + "="*80)
print("📈 COMPREHENSIVE PERFORMANCE METRICS FOR BOTH CLASSES (TUNED MODEL)")
print("="*80)

# Create detailed comparison dataframe for BOTH CLASSES
class_performance_data = []

for i, class_name in enumerate(class_names):
    class_performance_data.extend([
        {
            'Class': class_name,
            'Metric': 'Precision',
            'Training (10-Fold CV)': f"{train_precision[i]:.4f}",
            'Testing': f"{test_precision[i]:.4f}",
            'Difference': f"{train_precision[i]-test_precision[i]:+.4f}"
        },
        {
            'Class': class_name,
            'Metric': 'Recall',
            'Training (10-Fold CV)': f"{train_recall[i]:.4f}",
            'Testing': f"{test_recall[i]:.4f}",
            'Difference': f"{train_recall[i]-test_recall[i]:+.4f}"
        },
        {
            'Class': class_name,
            'Metric': 'F1-Score',
            'Training (10-Fold CV)': f"{train_f1[i]:.4f}",
            'Testing': f"{test_f1[i]:.4f}",
            'Difference': f"{train_f1[i]-test_f1[i]:+.4f}"
        }
    ])

# Add overall metrics
overall_metrics = [
    {
        'Class': 'Overall',
        'Metric': 'Accuracy',
        'Training (10-Fold CV)': f"{train_accuracy:.4f}",
        'Testing': f"{test_accuracy:.4f}",
        'Difference': f"{train_accuracy-test_accuracy:+.4f}"
    },
    {
        'Class': 'Overall',
        'Metric': 'ROC-AUC',
        'Training (10-Fold CV)': f"{train_roc_auc:.4f}",
        'Testing': f"{test_roc_auc:.4f}",
        'Difference': f"{train_roc_auc-test_roc_auc:+.4f}"
    },
    {
        'Class': 'Overall',
        'Metric': 'MCC',
        'Training (10-Fold CV)': f"{train_mcc:.4f}",
        'Testing': f"{test_mcc:.4f}",
        'Difference': f"{train_mcc-test_mcc:+.4f}"
    }
]

performance_df = pd.DataFrame(class_performance_data + overall_metrics)
print(performance_df.to_string(index=False))

# STEP 8: Enhanced Performance Visualization with ROC Curves
print("\n" + "="*80)
print("📊 GENERATING ENHANCED PERFORMANCE VISUALIZATIONS")
print("="*80)

# Create comprehensive performance comparison with ROC curves
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(20, 16))

# 1. Main Performance Comparison (Overall Metrics)
overall_metrics = ['Accuracy', 'ROC-AUC', 'MCC']
train_overall = [train_accuracy, train_roc_auc, train_mcc]
test_overall = [test_accuracy, test_roc_auc, test_mcc]

x = np.arange(len(overall_metrics))
width = 0.35

bars1 = ax1.bar(x - width/2, train_overall, width, label='Training (10-Fold CV)',
               color=elsevier_colors[0], alpha=0.8, edgecolor='black', linewidth=1.5)
bars2 = ax1.bar(x + width/2, test_overall, width, label='Testing (Held-Out)',
               color=elsevier_colors[1], alpha=0.8, edgecolor='black', linewidth=1.5)

# Add value labels on bars
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                f'{height:.3f}', ha='center', va='bottom',
                fontweight='bold', fontsize=11)

ax1.set_xlabel('Performance Metrics', fontsize=14, fontweight='bold')
ax1.set_ylabel('Score', fontsize=14, fontweight='bold')
ax1.set_title('Random Forest (Tuned) - Overall Performance\nTraining vs Testing',
              fontsize=16, fontweight='bold', pad=20)
ax1.set_xticks(x)
ax1.set_xticklabels(overall_metrics)
ax1.legend(fontsize=12, framealpha=0.9)
ax1.set_ylim(0, 1.1)
ax1.grid(True, alpha=0.3, linestyle='--')

# 2. ROC CURVE for Training and Testing Sets
ax2.plot(fpr_train, tpr_train, color=elsevier_colors[0], lw=2.5,
         label=f'Training ROC (AUC = {train_roc_auc:.3f})', alpha=0.8)
ax2.plot(fpr_test, tpr_test, color=elsevier_colors[1], lw=2.5,
         label=f'Testing ROC (AUC = {test_roc_auc:.3f})', alpha=0.8)

# Plot diagonal reference line
ax2.plot([0, 1], [0, 1], color='gray', lw=1.5, linestyle='--', alpha=0.7,
         label='Random Classifier (AUC = 0.5)')

# Fill area under curves
ax2.fill_between(fpr_train, tpr_train, alpha=0.2, color=elsevier_colors[0])
ax2.fill_between(fpr_test, tpr_test, alpha=0.2, color=elsevier_colors[1])

ax2.set_xlabel('False Positive Rate', fontsize=14, fontweight='bold')
ax2.set_ylabel('True Positive Rate', fontsize=14, fontweight='bold')
ax2.set_title('ROC Curves - Training vs Testing Sets\nRandom Forest (Tuned)',
              fontsize=16, fontweight='bold', pad=20)
ax2.legend(loc='lower right', fontsize=12, framealpha=0.9)
ax2.grid(True, alpha=0.3, linestyle='--')
ax2.set_xlim([0.0, 1.0])
ax2.set_ylim([0.0, 1.05])

# 3. Confusion Matrix - Training
im1 = ax3.imshow(train_cm, cmap='Blues', interpolation='nearest', alpha=0.8)
ax3.set_xlabel('Predicted Label', fontsize=14, fontweight='bold')
ax3.set_ylabel('True Label', fontsize=14, fontweight='bold')
ax3.set_title('Training Set - Confusion Matrix\n(10-Fold CV Average)',
              fontsize=16, fontweight='bold', pad=20)
ax3.set_xticks([0, 1])
ax3.set_yticks([0, 1])
ax3.set_xticklabels(class_names)
ax3.set_yticklabels(class_names)

# Add text annotations for training CM
for i in range(train_cm.shape[0]):
    for j in range(train_cm.shape[1]):
        ax3.text(j, i, f'{train_cm[i, j]}\n({train_cm[i, j]/np.sum(train_cm):.1%})',
                ha='center', va='center', fontsize=12, fontweight='bold',
                color='white' if train_cm[i, j] > np.max(train_cm)/2 else 'black')

# 4. Confusion Matrix - Testing
im2 = ax4.imshow(test_cm, cmap='Reds', interpolation='nearest', alpha=0.8)
ax4.set_xlabel('Predicted Label', fontsize=14, fontweight='bold')
ax4.set_ylabel('True Label', fontsize=14, fontweight='bold')
ax4.set_title('Testing Set - Confusion Matrix\n(Held-Out Dataset)',
              fontsize=16, fontweight='bold', pad=20)
ax4.set_xticks([0, 1])
ax4.set_yticks([0, 1])
ax4.set_xticklabels(class_names)
ax4.set_yticklabels(class_names)

# Add text annotations for testing CM
for i in range(test_cm.shape[0]):
    for j in range(test_cm.shape[1]):
        ax4.text(j, i, f'{test_cm[i, j]}\n({test_cm[i, j]/np.sum(test_cm):.1%})',
                ha='center', va='center', fontsize=12, fontweight='bold',
                color='white' if test_cm[i, j] > np.max(test_cm)/2 else 'black')

plt.tight_layout()
plt.savefig('random_forest_tuned_comprehensive_performance.png', dpi=300,
            bbox_inches='tight', facecolor='white')
plt.show()

# STEP 9: FIXED Classification Reports
print("\n" + "="*80)
print("📋 DETAILED CLASSIFICATION REPORTS FOR BOTH CLASSES")
print("="*80)

print("\nTRAINING SET CLASSIFICATION REPORT (10-Fold CV Average):")
training_report = classification_report(y_train_encoded, cv_predictions,
                                      target_names=class_names, digits=4)
print(training_report)

print("\nTESTING SET CLASSIFICATION REPORT (Held-Out):")
testing_report = classification_report(y_test_encoded, y_test_pred,
                                     target_names=class_names, digits=4)
print(testing_report)

# STEP 10: Individual Class Performance Analysis
print("\n" + "="*80)
print("🎯 INDIVIDUAL CLASS PERFORMANCE ANALYSIS")
print("="*80)

for i, class_name in enumerate(class_names):
    print(f"\n{class_name.upper()} CLASS PERFORMANCE:")
    print(f"  Training (10-Fold CV):")
    print(f"    Precision: {train_precision[i]:.4f}")
    print(f"    Recall:    {train_recall[i]:.4f}")
    print(f"    F1-Score:  {train_f1[i]:.4f}")
    print(f"  Testing (Held-Out):")
    print(f"    Precision: {test_precision[i]:.4f}")
    print(f"    Recall:    {test_recall[i]:.4f}")
    print(f"    F1-Score:  {test_f1[i]:.4f}")
    print(f"  Performance Gap:")
    print(f"    Precision: {train_precision[i]-test_precision[i]:+.4f}")
    print(f"    Recall:    {train_recall[i]-test_recall[i]:+.4f}")
    print(f"    F1-Score:  {train_f1[i]-test_f1[i]:+.4f}")

# STEP 11: Tuning Results Summary
print("\n" + "="*80)
print("🔬 HYPERPARAMETER TUNING SUMMARY")
print("="*80)

print(f"🎯 BEST PARAMETERS FOUND:")
for param, value in best_params.items():
    print(f"   {param}: {value}")

print(f"\n📊 PERFORMANCE COMPARISON:")
print(f"   Default Model Accuracy: {accuracy_default:.4f}")
print(f"   Tuned Model Accuracy:   {accuracy_tuned:.4f}")
print(f"   Improvement:            {improvement:+.4f}")

print(f"\n🚀 FINAL TUNED MODEL PERFORMANCE:")
print(f"   Training CV Accuracy: {train_accuracy:.4f}")
print(f"   Test Accuracy:        {test_accuracy:.4f}")
print(f"   Test ROC-AUC:         {test_roc_auc:.4f}")
print(f"   Test MCC:             {test_mcc:.4f}")

# STEP 12: Save ROC Data and Feature Importance (LIKE XGBOOST CODE)
print("\n" + "="*80)
print("💾 SAVING ROC DATA FOR COMBINED VISUALIZATIONS")
print("="*80)

# Save the tuned model
joblib.dump(best_rf_model, 'random_forest_tuned_model.pkl')
joblib.dump(label_encoder, 'random_forest_tuned_label_encoder.pkl')

# ============== SAVE ROC PREDICTION DATA ==============
print("\n💾 SAVING ROC PREDICTION DATA FILES...")

# File 1: Save NPZ file for ROC plotting
np.savez('random_forest_roc_predictions.npz',
         # Training set data (from cross-validation)
         y_train_true=y_train_encoded,
         y_train_prob=cv_probabilities[:, 1],  # Probability for positive class

         # Testing set data
         y_test_true=y_test_encoded,
         y_test_prob=y_test_proba[:, 1],  # Probability for positive class

         # Precomputed ROC curve points
         fpr_train=fpr_train,
         tpr_train=tpr_train,
         fpr_test=fpr_test,
         tpr_test=tpr_test,

         # AUC values
         train_auc=train_roc_auc,
         test_auc=test_roc_auc,

         # Metadata
         model_name='Random Forest',
         class_names=class_names,
         feature_count=len(feature_columns),
         best_params=str(best_params))
print("✅ Saved: random_forest_roc_predictions.npz")

# File 2: Save detailed predictions CSV
predictions_df = pd.DataFrame({
    'true_label': y_test_encoded,
    'predicted_label': y_test_pred,
    'probability_class_0': y_test_proba[:, 0],
    'probability_class_1': y_test_proba[:, 1],
    'correct': (y_test_encoded == y_test_pred)
})
predictions_df.to_csv('random_forest_predictions.csv', index=False)
print("✅ Saved: random_forest_predictions.csv")

# File 3: Save comprehensive metrics CSV
metrics_df = pd.DataFrame({
    'Model': ['Random Forest'],
    'Train_Accuracy': [train_accuracy],
    'Test_Accuracy': [test_accuracy],
    'Train_AUC': [train_roc_auc],
    'Test_AUC': [test_roc_auc],
    'Train_Precision_0': [train_precision[0]],
    'Train_Precision_1': [train_precision[1]],
    'Test_Precision_0': [test_precision[0]],
    'Test_Precision_1': [test_precision[1]],
    'Train_Recall_0': [train_recall[0]],
    'Train_Recall_1': [train_recall[1]],
    'Test_Recall_0': [test_recall[0]],
    'Test_Recall_1': [test_recall[1]],
    'Train_F1_0': [train_f1[0]],
    'Train_F1_1': [train_f1[1]],
    'Test_F1_0': [test_f1[0]],
    'Test_F1_1': [test_f1[1]],
    'Train_MCC': [train_mcc],
    'Test_MCC': [test_mcc],
    'Num_Features': [len(feature_columns)],
    'N_Estimators': [best_params.get('n_estimators', 'N/A')],
    'Max_Depth': [best_params.get('max_depth', 'N/A')],
    'Min_Samples_Split': [best_params.get('min_samples_split', 'N/A')],
    'Min_Samples_Leaf': [best_params.get('min_samples_leaf', 'N/A')]
})
metrics_df.to_csv('random_forest_metrics.csv', index=False)
print("✅ Saved: random_forest_metrics.csv")

# File 4: Save all predictions (training + testing) for reference
all_predictions_df = pd.DataFrame({
    'dataset': ['train'] * len(y_train_encoded) + ['test'] * len(y_test_encoded),
    'true_label': np.concatenate([y_train_encoded, y_test_encoded]),
    'predicted_label': np.concatenate([cv_predictions, y_test_pred]),
    'probability_class_1': np.concatenate([cv_probabilities[:, 1], y_test_proba[:, 1]])
})
all_predictions_df.to_csv('random_forest_all_predictions.csv', index=False)
print("✅ Saved: random_forest_all_predictions.csv")

# File 5: Save feature importance (Random Forest specific)
feature_importances = best_rf_model.feature_importances_
importance_df = pd.DataFrame({
    'Feature': feature_columns,
    'Importance': feature_importances
}).sort_values('Importance', ascending=False)

# Save top 30 features for visualization
top_features = importance_df.head(30)
importance_df.to_excel('random_forest_feature_importances.xlsx', index=False)
top_features.to_excel('random_forest_top30_features.xlsx', index=False)
print("✅ Saved: random_forest_feature_importances.xlsx")
print("✅ Saved: random_forest_top30_features.xlsx")

# Create feature importance visualization
plt.figure(figsize=(12, 8))
plt.barh(range(len(top_features)), top_features['Importance'][::-1],
         color=elsevier_colors[0], alpha=0.8, edgecolor='black')
plt.yticks(range(len(top_features)), top_features['Feature'][::-1], fontsize=10)
plt.xlabel('Feature Importance Score', fontsize=14, fontweight='bold')
plt.ylabel('Feature', fontsize=14, fontweight='bold')
plt.title('Random Forest - Top 30 Feature Importances', fontsize=16, fontweight='bold', pad=20)
plt.grid(True, alpha=0.3, linestyle='--', axis='x')
plt.tight_layout()
plt.savefig('random_forest_feature_importances.png', dpi=300, bbox_inches='tight', facecolor='white')
plt.show()

# File 6: Save grid search results
grid_results_df = pd.DataFrame(grid_search.cv_results_)
grid_results_df.to_excel('random_forest_grid_search_results.xlsx', index=False)
print("✅ Saved: random_forest_grid_search_results.xlsx")

# File 7: Save tuning parameters
tuning_params = {
    'best_params': best_params,
    'best_cv_score': grid_search.best_score_,
    'default_accuracy': accuracy_default,
    'tuned_accuracy': accuracy_tuned,
    'improvement': improvement,
    'test_roc_auc': test_roc_auc,
    'test_mcc': test_mcc
}
tuning_params_df = pd.DataFrame([tuning_params])
tuning_params_df.to_excel('random_forest_tuning_parameters.xlsx', index=False)

print(f"\n🎯 RANDOM FOREST WITH HYPERPARAMETER TUNING COMPLETED!")
print(f"📊 Training (10-Fold CV) Accuracy: {train_accuracy:.1%}")
print(f"📊 Testing Accuracy: {test_accuracy:.1%}")
print(f"📈 Training ROC-AUC: {train_roc_auc:.3f}")
print(f"📈 Testing ROC-AUC: {test_roc_auc:.3f}")
print(f"⚡ Improvement over default: {improvement:+.2%}")
print(f"🔍 Performance metrics shown for BOTH classes: {class_names}")

print(f"\n📁 FILES GENERATED FOR ROC CURVES:")
print(f"   1. random_forest_roc_predictions.npz     - Main ROC data file")
print(f"   2. random_forest_predictions.csv         - Detailed test predictions")
print(f"   3. random_forest_metrics.csv             - Performance metrics")
print(f"   4. random_forest_all_predictions.csv     - All predictions (train+test)")
print(f"   5. random_forest_feature_importances.xlsx - Feature importance scores")
print(f"   6. random_forest_top30_features.xlsx     - Top 30 features")
print(f"   7. random_forest_tuned_model.pkl         - Trained model")
print(f"   8. random_forest_grid_search_results.xlsx - Grid search results")

print(f"\n🎯 Use 'random_forest_roc_predictions.npz' with other model files for combined ROC plots!")
print("🚀 TUNED RANDOM FOREST MODEL READY FOR DEPLOYMENT!")

### **Support Vector Machine:**

Requirements:
training-set-selected-features.xlsx,
testing-set-selected-features.xlsx

In [ ]:
# Install Intel optimizations (run this once)
!pip install scikit-learn-intelex

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import time

# Apply Intel optimizations at the VERY BEGINNING
from sklearnex import patch_sklearn
patch_sklearn()

# Now import sklearn modules - they will be Intel-optimized
from sklearn.svm import SVC
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                           roc_auc_score, roc_curve, matthews_corrcoef, confusion_matrix,
                           classification_report)
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import cross_val_score, StratifiedKFold, cross_val_predict, GridSearchCV
from sklearn.pipeline import Pipeline

print("🚀 ENHANCED SVM WITH INTEL OPTIMIZATIONS & EXPANDED PARAMETERS (10-FOLD CV)")
print("="*80)
print("⚡ Intel-optimized scikit-learn + Expanded parameter grid")
print("="*80)

# Set publication-quality parameters
plt.rcParams['font.size'] = 12
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['axes.titlesize'] = 16
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
plt.rcParams['legend.fontsize'] = 12
plt.rcParams['figure.titlesize'] = 18
plt.rcParams['font.family'] = ['DejaVu Sans', 'Arial', 'Helvetica', 'sans-serif']

# Professional Elsevier color scheme
elsevier_colors = ['#2E86AB', '#A23B72', '#F18F01', '#C73E1D', '#1A936F', '#114B5F']

# STEP 1: Load the feature-selected datasets
print("📊 Loading feature-selected datasets...")

train_df = pd.read_excel('training_set_selected_features.xlsx')
test_df = pd.read_excel('testing_set_selected_features.xlsx')

print(f"Training set: {train_df.shape}")
print(f"Testing set: {test_df.shape}")

# STEP 2: Prepare features and target
print("\n" + "="*80)
print("🔧 PREPARING DATA")
print("="*80)

# Identify feature columns (exclude metadata)
metadata_columns = ['COMPOUND ID', 'SMILE CODE', 'Ligand Type']
feature_columns = [col for col in train_df.columns if col not in metadata_columns]

X_train = train_df[feature_columns]
y_train = train_df['Ligand Type']

X_test = test_df[feature_columns]
y_test = test_df['Ligand Type']

print(f"Selected features: {len(feature_columns)}")
print(f"Training samples: {X_train.shape[0]}")
print(f"Testing samples: {X_test.shape[0]}")

# Handle missing values if any
if X_train.isnull().sum().sum() > 0:
    print("   ⚠️  Missing values found - imputing with mean...")
    X_train = X_train.fillna(X_train.mean())
    X_test = X_test.fillna(X_test.mean())

# Encode labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

class_names = [str(cls) for cls in label_encoder.classes_]
print(f"Class names: {class_names}")

print("✅ Data prepared successfully!")

# STEP 3: ENHANCED Hyperparameter Tuning with EXPANDED PARAMETERS + INTEL OPTIMIZATIONS
print("\n" + "="*80)
print("🎯 ENHANCED HYPERPARAMETER TUNING WITH EXPANDED GRID (10-FOLD CV)")
print("⚡ Intel-optimized + More parameters + Faster execution")
print("="*80)

# EXPANDED parameter grid - more comprehensive but still manageable
enhanced_param_grid = {
    'svm__C': [0.01, 0.1, 1, 10, 100],           # Expanded from 3 to 5 values
    'svm__kernel': ['linear', 'rbf', 'poly'],     # Added back 'poly' kernel
    'svm__gamma': ['scale', 'auto', 0.01, 0.1, 1], # Expanded from 2 to 5 values
    'svm__degree': [2, 3]                         # Added for poly kernel
}

# Create pipeline with Intel-optimized SVM
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('svm', SVC(
        probability=True,
        random_state=42,
        max_iter=5000,    # Prevent endless runs
        tol=1e-3          # Faster convergence
    ))
])

# 10-fold cross-validation strategy
cv_strategy = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Calculate enhanced metrics
total_combinations = (len(enhanced_param_grid['svm__C']) *
                     len(enhanced_param_grid['svm__kernel']) *
                     len(enhanced_param_grid['svm__gamma']) *
                     len(enhanced_param_grid['svm__degree']))
total_fits = total_combinations * 10

print("🔍 Performing ENHANCED Grid Search with 10-fold CV...")
print(f"   ENHANCED Parameter grid: {enhanced_param_grid}")
print(f"   Total parameter combinations: {total_combinations}")
print(f"   Total fits to perform: {total_fits}")
print(f"   ⚡ Intel optimizations make this feasible despite larger grid!")
print(f"   🎯 More comprehensive search for better performance!")

# GridSearchCV with parallel processing
grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=enhanced_param_grid,
    cv=cv_strategy,
    scoring='accuracy',
    n_jobs=-1,           # Use all available cores
    verbose=1,
    return_train_score=True
)

# Perform ENHANCED grid search with timing
print(f"\n🕒 Starting ENHANCED grid search at {time.strftime('%H:%M:%S')}")
start_time = time.time()

grid_search.fit(X_train, y_train_encoded)

end_time = time.time()
execution_time = end_time - start_time

print(f"✅ ENHANCED Grid Search completed in {execution_time:.2f} seconds ({execution_time/60:.2f} minutes)!")
print("⚡ Intel optimizations + expanded parameters successfully executed!")

# Display best parameters and scores
print(f"\n🎯 BEST PARAMETERS FOUND:")
best_params = grid_search.best_params_
for param, value in best_params.items():
    print(f"   {param}: {value}")
print(f"   Best CV Score (10-fold): {grid_search.best_score_:.4f}")

# Analyze grid search results
print(f"\n📊 GRID SEARCH ANALYSIS:")
results_df = pd.DataFrame(grid_search.cv_results_)
top_5_results = results_df.nlargest(5, 'mean_test_score')[['params', 'mean_test_score', 'std_test_score']]
print("Top 5 parameter combinations:")
for i, (idx, row) in enumerate(top_5_results.iterrows()):
    print(f"   {i+1}. Score: {row['mean_test_score']:.4f} ± {row['std_test_score']:.4f}")
    print(f"      Params: {row['params']}")

# STEP 4: Compare Default vs Enhanced Tuned Models
print("\n" + "="*80)
print("📊 COMPARISON: DEFAULT vs ENHANCED TUNED MODEL")
print("="*80)

# Train model with default parameters for comparison
default_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('svm', SVC(probability=True, random_state=42, max_iter=5000, tol=1e-3))
])

default_start = time.time()
default_pipeline.fit(X_train, y_train_encoded)
default_end = time.time()
y_pred_default = default_pipeline.predict(X_test)
accuracy_default = accuracy_score(y_test_encoded, y_pred_default)

# Use best model from enhanced grid search
best_svm_model = grid_search.best_estimator_
tuned_start = time.time()
y_pred_tuned = best_svm_model.predict(X_test)
tuned_end = time.time()
accuracy_tuned = accuracy_score(y_test_encoded, y_pred_tuned)

improvement = accuracy_tuned - accuracy_default
default_time = default_end - default_start
tuned_time = tuned_end - tuned_start

print(f"   Default Parameters Accuracy: {accuracy_default:.4f}")
print(f"   Enhanced Tuned Accuracy:     {accuracy_tuned:.4f}")
print(f"   Improvement:                 {improvement:+.4f}")
print(f"   Default training time: {default_time:.2f}s")
print(f"   Tuned prediction time: {tuned_time:.2f}s")

if improvement > 0:
    print(f"   ✅ Enhanced tuning improved accuracy by {improvement:.2%}")
else:
    print(f"   ⚠️  Enhanced tuning did not improve accuracy")

# STEP 5: 10-Fold Cross-Validation with Enhanced Tuned Model
print("\n" + "="*80)
print("🎯 10-FOLD CROSS-VALIDATION WITH ENHANCED TUNED PARAMETERS")
print("="*80)

print("🚀 Performing 10-fold cross-validation with enhanced tuned parameters...")

cv_start = time.time()

# Get cross-validated predictions
cv_predictions = cross_val_predict(best_svm_model, X_train, y_train_encoded,
                                 cv=cv_strategy, method='predict', n_jobs=-1)
cv_probabilities = cross_val_predict(best_svm_model, X_train, y_train_encoded,
                                   cv=cv_strategy, method='predict_proba', n_jobs=-1)

cv_end = time.time()

print(f"✅ 10-fold cross-validation completed in {cv_end - cv_start:.2f} seconds!")

# Calculate training metrics
train_accuracy = accuracy_score(y_train_encoded, cv_predictions)
train_precision = precision_score(y_train_encoded, cv_predictions, average=None)
train_recall = recall_score(y_train_encoded, cv_predictions, average=None)
train_f1 = f1_score(y_train_encoded, cv_predictions, average=None)
train_roc_auc = roc_auc_score(y_train_encoded, cv_probabilities[:, 1])
train_mcc = matthews_corrcoef(y_train_encoded, cv_predictions)

# Individual fold accuracies
cv_scores = cross_val_score(best_svm_model, X_train, y_train_encoded,
                          cv=cv_strategy, scoring='accuracy', n_jobs=-1)

print(f"\n📊 10-FOLD CROSS-VALIDATION RESULTS (TRAINING PERFORMANCE):")
print(f"   Fold scores: {[f'{score:.4f}' for score in cv_scores]}")
print(f"   Mean CV Accuracy: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")
print(f"   Standard Deviation: {cv_scores.std():.4f}")

# STEP 6: Make predictions on TEST set with enhanced tuned model
print("\n" + "="*80)
print("📊 MODEL EVALUATION WITH ENHANCED TUNED PARAMETERS")
print("="*80)

# Predictions on TEST set
y_test_pred = best_svm_model.predict(X_test)
y_test_proba = best_svm_model.predict_proba(X_test)

# Calculate metrics for TESTING set
test_accuracy = accuracy_score(y_test_encoded, y_test_pred)
test_precision = precision_score(y_test_encoded, y_test_pred, average=None)
test_recall = recall_score(y_test_encoded, y_test_pred, average=None)
test_f1 = f1_score(y_test_encoded, y_test_pred, average=None)
test_roc_auc = roc_auc_score(y_test_encoded, y_test_proba[:, 1])
test_mcc = matthews_corrcoef(y_test_encoded, y_test_pred)

# Confusion matrices
train_cm = confusion_matrix(y_train_encoded, cv_predictions)
test_cm = confusion_matrix(y_test_encoded, y_test_pred)

# STEP 7: Display comprehensive results
print("\n" + "="*80)
print("📈 COMPREHENSIVE PERFORMANCE METRICS (ENHANCED TUNED MODEL)")
print("="*80)

# Create detailed comparison dataframe
class_performance_data = []

for i, class_name in enumerate(class_names):
    class_performance_data.extend([
        {
            'Class': class_name,
            'Metric': 'Precision',
            'Training (10-Fold CV)': f"{train_precision[i]:.4f}",
            'Testing': f"{test_precision[i]:.4f}",
            'Difference': f"{train_precision[i]-test_precision[i]:+.4f}"
        },
        {
            'Class': class_name,
            'Metric': 'Recall',
            'Training (10-Fold CV)': f"{train_recall[i]:.4f}",
            'Testing': f"{test_recall[i]:.4f}",
            'Difference': f"{train_recall[i]-test_recall[i]:+.4f}"
        },
        {
            'Class': class_name,
            'Metric': 'F1-Score',
            'Training (10-Fold CV)': f"{train_f1[i]:.4f}",
            'Testing': f"{test_f1[i]:.4f}",
            'Difference': f"{train_f1[i]-test_f1[i]:+.4f}"
        }
    ])

# Add overall metrics
overall_metrics = [
    {
        'Class': 'Overall',
        'Metric': 'Accuracy',
        'Training (10-Fold CV)': f"{train_accuracy:.4f}",
        'Testing': f"{test_accuracy:.4f}",
        'Difference': f"{train_accuracy-test_accuracy:+.4f}"
    },
    {
        'Class': 'Overall',
        'Metric': 'ROC-AUC',
        'Training (10-Fold CV)': f"{train_roc_auc:.4f}",
        'Testing': f"{test_roc_auc:.4f}",
        'Difference': f"{train_roc_auc-test_roc_auc:+.4f}"
    },
    {
        'Class': 'Overall',
        'Metric': 'MCC',
        'Training (10-Fold CV)': f"{train_mcc:.4f}",
        'Testing': f"{test_mcc:.4f}",
        'Difference': f"{train_mcc-test_mcc:+.4f}"
    }
]

performance_df = pd.DataFrame(class_performance_data + overall_metrics)
print(performance_df.to_string(index=False))

# STEP 8: Enhanced Performance Visualization
print("\n" + "="*80)
print("📊 GENERATING ENHANCED PERFORMANCE VISUALIZATIONS")
print("="*80)

# Create comprehensive performance comparison
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(20, 16))

# 1. Main Performance Comparison
overall_metrics = ['Accuracy', 'ROC-AUC', 'MCC']
train_overall = [train_accuracy, train_roc_auc, train_mcc]
test_overall = [test_accuracy, test_roc_auc, test_mcc]

x = np.arange(len(overall_metrics))
width = 0.35

bars1 = ax1.bar(x - width/2, train_overall, width, label='Training (10-Fold CV)',
               color=elsevier_colors[0], alpha=0.8, edgecolor='black', linewidth=1.5)
bars2 = ax1.bar(x + width/2, test_overall, width, label='Testing (Held-Out)',
               color=elsevier_colors[1], alpha=0.8, edgecolor='black', linewidth=1.5)

for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                f'{height:.3f}', ha='center', va='bottom',
                fontweight='bold', fontsize=11)

ax1.set_xlabel('Performance Metrics', fontsize=14, fontweight='bold')
ax1.set_ylabel('Score', fontsize=14, fontweight='bold')
ax1.set_title('ENHANCED SVM (Tuned) - Overall Performance\nTraining vs Testing',
              fontsize=16, fontweight='bold', pad=20)
ax1.set_xticks(x)
ax1.set_xticklabels(overall_metrics)
ax1.legend(fontsize=12, framealpha=0.9)
ax1.set_ylim(0, 1.1)
ax1.grid(True, alpha=0.3, linestyle='--')

# 2. Class-wise Precision Comparison
class_metrics = ['Precision', 'Recall', 'F1-Score']
x_class = np.arange(len(class_metrics))
width_class = 0.25

for i, class_name in enumerate(class_names):
    offset = width_class * (i - 0.5)
    class_train_scores = [train_precision[i], train_recall[i], train_f1[i]]
    class_test_scores = [test_precision[i], test_recall[i], test_f1[i]]

    ax2.bar(x_class + offset, class_train_scores, width_class,
            label=f'{class_name} (Train)', alpha=0.7,
            color=elsevier_colors[i*2], edgecolor='black')
    ax2.bar(x_class + offset + width_class, class_test_scores, width_class,
            label=f'{class_name} (Test)', alpha=0.7,
            color=elsevier_colors[i*2+1], edgecolor='black')

ax2.set_xlabel('Metrics', fontsize=14, fontweight='bold')
ax2.set_ylabel('Score', fontsize=14, fontweight='bold')
ax2.set_title('ENHANCED SVM (Tuned) - Class-wise Performance\nTraining vs Testing',
              fontsize=16, fontweight='bold', pad=20)
ax2.set_xticks(x_class + width_class/2)
ax2.set_xticklabels(class_metrics)
ax2.legend(fontsize=10, framealpha=0.9, ncol=2)
ax2.set_ylim(0, 1.1)
ax2.grid(True, alpha=0.3, linestyle='--')

# 3. Confusion Matrix - Training
im1 = ax3.imshow(train_cm, cmap='Blues', interpolation='nearest', alpha=0.8)
ax3.set_xlabel('Predicted Label', fontsize=14, fontweight='bold')
ax3.set_ylabel('True Label', fontsize=14, fontweight='bold')
ax3.set_title('Training Set - Confusion Matrix\n(10-Fold CV Average)',
              fontsize=16, fontweight='bold', pad=20)
ax3.set_xticks([0, 1])
ax3.set_yticks([0, 1])
ax3.set_xticklabels(class_names)
ax3.set_yticklabels(class_names)

for i in range(train_cm.shape[0]):
    for j in range(train_cm.shape[1]):
        ax3.text(j, i, f'{train_cm[i, j]}\n({train_cm[i, j]/np.sum(train_cm):.1%})',
                ha='center', va='center', fontsize=12, fontweight='bold',
                color='white' if train_cm[i, j] > np.max(train_cm)/2 else 'black')

# 4. Confusion Matrix - Testing
im2 = ax4.imshow(test_cm, cmap='Reds', interpolation='nearest', alpha=0.8)
ax4.set_xlabel('Predicted Label', fontsize=14, fontweight='bold')
ax4.set_ylabel('True Label', fontsize=14, fontweight='bold')
ax4.set_title('Testing Set - Confusion Matrix\n(Held-Out Dataset)',
              fontsize=16, fontweight='bold', pad=20)
ax4.set_xticks([0, 1])
ax4.set_yticks([0, 1])
ax4.set_xticklabels(class_names)
ax4.set_yticklabels(class_names)

for i in range(test_cm.shape[0]):
    for j in range(test_cm.shape[1]):
        ax4.text(j, i, f'{test_cm[i, j]}\n({test_cm[i, j]/np.sum(test_cm):.1%})',
                ha='center', va='center', fontsize=12, fontweight='bold',
                color='white' if test_cm[i, j] > np.max(test_cm)/2 else 'black')

plt.tight_layout()
plt.savefig('enhanced_svm_tuned_comprehensive_performance.png', dpi=300,
            bbox_inches='tight', facecolor='white')
plt.show()

# STEP 9: Classification Reports
print("\n" + "="*80)
print("📋 DETAILED CLASSIFICATION REPORTS")
print("="*80)

print("\nTRAINING SET CLASSIFICATION REPORT (10-Fold CV Average):")
training_report = classification_report(y_train_encoded, cv_predictions,
                                      target_names=class_names, digits=4)
print(training_report)

print("\nTESTING SET CLASSIFICATION REPORT (Held-Out):")
testing_report = classification_report(y_test_encoded, y_test_pred,
                                     target_names=class_names, digits=4)
print(testing_report)

# STEP 10: Enhanced Tuning Results Summary
print("\n" + "="*80)
print("🔬 ENHANCED HYPERPARAMETER TUNING SUMMARY")
print("="*80)

print(f"🎯 BEST PARAMETERS FOUND:")
for param, value in best_params.items():
    print(f"   {param}: {value}")

print(f"\n📊 PERFORMANCE COMPARISON:")
print(f"   Default Model Accuracy: {accuracy_default:.4f}")
print(f"   Enhanced Tuned Accuracy: {accuracy_tuned:.4f}")
print(f"   Improvement:            {improvement:+.4f}")

print(f"\n⚡ ENHANCEMENTS APPLIED:")
print(f"   • Intel optimizations: 2-10x speedup")
print(f"   • Expanded parameter grid: {total_combinations} combinations")
print(f"   • Added 'poly' kernel with degree parameter")
print(f"   • More C values: [0.01, 0.1, 1, 10, 100]")
print(f"   • More gamma values: ['scale', 'auto', 0.01, 0.1, 1]")
print(f"   • Total execution time: {execution_time:.2f}s")

print(f"\n🚀 FINAL ENHANCED TUNED MODEL PERFORMANCE:")
print(f"   Training CV Accuracy: {train_accuracy:.4f}")
print(f"   Test Accuracy:        {test_accuracy:.4f}")
print(f"   Test ROC-AUC:         {test_roc_auc:.4f}")
print(f"   Test MCC:             {test_mcc:.4f}")

# STEP 11: Save enhanced model and results
print("\n" + "="*80)
print("💾 SAVING ENHANCED MODEL AND RESULTS")
print("="*80)

# Save the enhanced model
joblib.dump(best_svm_model, 'enhanced_svm_tuned_model.pkl')
joblib.dump(label_encoder, 'enhanced_svm_tuned_label_encoder.pkl')

# Save predictions with probabilities
test_results_df = test_df.copy()
test_results_df['Predicted_Class'] = label_encoder.inverse_transform(y_test_pred)
test_results_df['Prediction_Probability_0'] = y_test_proba[:, 0]
test_results_df['Prediction_Probability_1'] = y_test_proba[:, 1]
test_results_df['Prediction_Correct'] = (y_test_encoded == y_test_pred)

test_results_df.to_excel('enhanced_svm_tuned_testing_predictions.xlsx', index=False)

# Save enhanced tuning parameters
tuning_params = {
    'best_params': best_params,
    'best_cv_score': grid_search.best_score_,
    'default_accuracy': accuracy_default,
    'tuned_accuracy': accuracy_tuned,
    'improvement': improvement,
    'test_roc_auc': test_roc_auc,
    'test_mcc': test_mcc,
    'enhancements': [
        'Intel optimizations (2-10x speedup)',
        'Expanded parameter grid',
        'Added poly kernel with degree',
        'More C and gamma values',
        '10-fold cross-validation'
    ],
    'execution_time_seconds': execution_time,
    'total_combinations': total_combinations
}
tuning_params_df = pd.DataFrame([tuning_params])
tuning_params_df.to_excel('enhanced_svm_tuning_parameters.xlsx', index=False)

# Save grid search results
grid_results_df = pd.DataFrame(grid_search.cv_results_)
grid_results_df.to_excel('enhanced_svm_grid_search_results.xlsx', index=False)

# Save top 10 parameter combinations
top_10_results = results_df.nlargest(10, 'mean_test_score')[
    ['mean_test_score', 'std_test_score', 'params']
]
top_10_results.to_excel('enhanced_svm_top_10_parameters.xlsx', index=False)

# ============== ADD THIS SECTION: SAVE ROC DATA ==============
print("\n" + "="*80)
print("💾 SAVING ROC DATA FOR COMBINED VISUALIZATIONS")
print("="*80)

# Compute ROC curve points for SVM (required for NPZ file)
fpr_train_svm, tpr_train_svm, _ = roc_curve(y_train_encoded, cv_probabilities[:, 1])
fpr_test_svm, tpr_test_svm, _ = roc_curve(y_test_encoded, y_test_proba[:, 1])

print("\n💾 SAVING ROC PREDICTION DATA FILES...")

# File 1: Save NPZ file for ROC plotting (MATCHES XGBoost FORMAT)
np.savez('enhanced_svm_roc_predictions.npz',
         # Training set data (from cross-validation)
         y_train_true=y_train_encoded,
         y_train_prob=cv_probabilities[:, 1],  # Probability for positive class

         # Testing set data
         y_test_true=y_test_encoded,
         y_test_prob=y_test_proba[:, 1],  # Probability for positive class

         # Precomputed ROC curve points
         fpr_train=fpr_train_svm,
         tpr_train=tpr_train_svm,
         fpr_test=fpr_test_svm,
         tpr_test=tpr_test_svm,

         # AUC values
         train_auc=train_roc_auc,
         test_auc=test_roc_auc,

         # Metadata
         model_name='Enhanced_SVM',
         class_names=class_names,
         feature_count=len(feature_columns),
         best_params=str(best_params))
print("✅ Saved: enhanced_svm_roc_predictions.npz")

# File 2: Save detailed predictions CSV
predictions_df = pd.DataFrame({
    'true_label': y_test_encoded,
    'predicted_label': y_test_pred,
    'probability_class_0': y_test_proba[:, 0],
    'probability_class_1': y_test_proba[:, 1],
    'correct': (y_test_encoded == y_test_pred)
})
predictions_df.to_csv('enhanced_svm_predictions.csv', index=False)
print("✅ Saved: enhanced_svm_predictions.csv")

# File 3: Save comprehensive metrics CSV
metrics_df = pd.DataFrame({
    'Model': ['Enhanced_SVM'],
    'Train_Accuracy': [train_accuracy],
    'Test_Accuracy': [test_accuracy],
    'Train_AUC': [train_roc_auc],
    'Test_AUC': [test_roc_auc],
    'Train_Precision_0': [train_precision[0]],
    'Train_Precision_1': [train_precision[1]],
    'Test_Precision_0': [test_precision[0]],
    'Test_Precision_1': [test_precision[1]],
    'Train_Recall_0': [train_recall[0]],
    'Train_Recall_1': [train_recall[1]],
    'Test_Recall_0': [test_recall[0]],
    'Test_Recall_1': [test_recall[1]],
    'Train_F1_0': [train_f1[0]],
    'Train_F1_1': [train_f1[1]],
    'Test_F1_0': [test_f1[0]],
    'Test_F1_1': [test_f1[1]],
    'Train_MCC': [train_mcc],
    'Test_MCC': [test_mcc],
    'Num_Features': [len(feature_columns)],
    'SVM_C': [best_params.get('svm__C', 'N/A')],
    'SVM_Kernel': [best_params.get('svm__kernel', 'N/A')],
    'SVM_Gamma': [best_params.get('svm__gamma', 'N/A')]
})
metrics_df.to_csv('enhanced_svm_metrics.csv', index=False)
print("✅ Saved: enhanced_svm_metrics.csv")

# File 4: Save all predictions (training + testing) for reference
all_predictions_df = pd.DataFrame({
    'dataset': ['train'] * len(y_train_encoded) + ['test'] * len(y_test_encoded),
    'true_label': np.concatenate([y_train_encoded, y_test_encoded]),
    'predicted_label': np.concatenate([cv_predictions, y_test_pred]),
    'probability_class_1': np.concatenate([cv_probabilities[:, 1], y_test_proba[:, 1]])
})
all_predictions_df.to_csv('enhanced_svm_all_predictions.csv', index=False)
print("✅ Saved: enhanced_svm_all_predictions.csv")

print(f"\n📁 NEW ROC DATA FILES GENERATED:")
print(f"   1. enhanced_svm_roc_predictions.npz - Main ROC data file (matches XGBoost format)")
print(f"   2. enhanced_svm_predictions.csv     - Detailed test predictions")
print(f"   3. enhanced_svm_metrics.csv         - Performance metrics")
print(f"   4. enhanced_svm_all_predictions.csv - All predictions (train+test)")
print(f"🎯 Use 'enhanced_svm_roc_predictions.npz' with other model files for combined ROC plots!")
# ============== END OF ADDED SECTION ==============

print("✅ ENHANCED FILES SAVED:")
print(f"   • enhanced_svm_tuned_model.pkl")
print(f"   • enhanced_svm_tuned_label_encoder.pkl")
print(f"   • enhanced_svm_tuned_testing_predictions.xlsx")
print(f"   • enhanced_svm_tuning_parameters.xlsx")
print(f"   • enhanced_svm_grid_search_results.xlsx")
print(f"   • enhanced_svm_top_10_parameters.xlsx")
print(f"   • enhanced_svm_tuned_comprehensive_performance.png")

print(f"\n🎯 ENHANCED SUPPORT VECTOR MACHINE COMPLETED!")
print(f"⚡ Intel optimizations + Expanded parameters applied!")
print(f"📊 Training (10-Fold CV) Accuracy: {train_accuracy:.1%}")
print(f"📊 Testing Accuracy: {test_accuracy:.1%}")
print(f"📈 ROC-AUC: {test_roc_auc:.3f}")
print(f"⚡ Improvement over default: {improvement:+.2%}")
print(f"🔍 Comprehensive parameter search completed!")
print("🚀 ENHANCED TUNED MODEL READY FOR DEPLOYMENT!")

### **Multi-Layer Perceptron / MLP Classifier:**

Requirements: training-set-selected-features.xlsx, testing-set-selected-features.xlsx

In [ ]:
import pandas as pd
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                           roc_auc_score, roc_curve, matthews_corrcoef, confusion_matrix,
                           classification_report)
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import cross_val_score, StratifiedKFold, cross_val_predict, GridSearchCV
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

# Set publication-quality parameters (Fixed font settings)
plt.rcParams['font.size'] = 12
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['axes.titlesize'] = 16
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
plt.rcParams['legend.fontsize'] = 12
plt.rcParams['figure.titlesize'] = 18

# Use available system fonts to avoid warnings
plt.rcParams['font.family'] = ['DejaVu Sans', 'Arial', 'Helvetica', 'sans-serif']

# Professional Elsevier color scheme
elsevier_colors = ['#2E86AB', '#A23B72', '#F18F01', '#C73E1D', '#1A936F', '#114B5F']

print("🚀 MLP CLASSIFIER WITH HYPERPARAMETER TUNING (10-FOLD CV + GRID SEARCH)")
print("="*80)

# STEP 1: Load the feature-selected datasets
print("📊 Loading feature-selected datasets...")

train_df = pd.read_excel('training_set_selected_features.xlsx')
test_df = pd.read_excel('testing_set_selected_features.xlsx')

print(f"Training set: {train_df.shape}")
print(f"Testing set: {test_df.shape}")

# STEP 2: Prepare features and target
print("\n" + "="*80)
print("🔧 PREPARING DATA")
print("="*80)

# Identify feature columns (exclude metadata)
metadata_columns = ['COMPOUND ID', 'SMILE CODE', 'Ligand Type']
feature_columns = [col for col in train_df.columns if col not in metadata_columns]

X_train = train_df[feature_columns]
y_train = train_df['Ligand Type']

X_test = test_df[feature_columns]
y_test = test_df['Ligand Type']

print(f"Selected features: {len(feature_columns)}")
print(f"Training samples: {X_train.shape[0]}")
print(f"Testing samples: {X_test.shape[0]}")

# Encode labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# FIXED: Properly get class names as strings
class_names = [str(cls) for cls in label_encoder.classes_]
print(f"Class names: {class_names}")

# Scale features for neural network (important for MLP)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("✅ Data prepared and scaled successfully!")

# STEP 3: Hyperparameter Tuning with GridSearchCV and 10-Fold CV
print("\n" + "="*80)
print("🎯 HYPERPARAMETER TUNING WITH GRIDSEARCHCV (10-FOLD CV)")
print("="*80)

# Define parameter grid for MLPClassifier
param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50)],
    'activation': ['relu', 'tanh'],
    'alpha': [0.0001, 0.001, 0.01],
    'learning_rate': ['constant', 'adaptive'],
    'learning_rate_init': [0.001, 0.01],
    'max_iter': [500, 1000]
}

# Initialize MLPClassifier
mlp = MLPClassifier(random_state=42, early_stopping=True, validation_fraction=0.1)

# 10-fold cross-validation strategy
cv_strategy = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# GridSearchCV with 10-fold CV
print("🔍 Performing Grid Search with 10-fold CV...")
print(f"   Parameter grid: {param_grid}")
print(f"   Total parameter combinations: {len(param_grid['hidden_layer_sizes']) * len(param_grid['activation']) * len(param_grid['alpha']) * len(param_grid['learning_rate']) * len(param_grid['learning_rate_init']) * len(param_grid['max_iter'])}")
print(f"   This may take a while...")

grid_search = GridSearchCV(
    estimator=mlp,
    param_grid=param_grid,
    cv=cv_strategy,
    scoring='accuracy',
    n_jobs=-1,
    verbose=1
)

# Perform grid search
grid_search.fit(X_train_scaled, y_train_encoded)

print("✅ Grid Search completed!")

# Display best parameters and scores
print(f"\n🎯 BEST PARAMETERS FOUND:")
best_params = grid_search.best_params_
for param, value in best_params.items():
    print(f"   {param}: {value}")
print(f"   Best CV Score: {grid_search.best_score_:.4f}")

# STEP 4: Compare Default vs Tuned Models
print("\n" + "="*80)
print("📊 COMPARISON: DEFAULT vs TUNED PARAMETERS")
print("="*80)

# Train model with default parameters for comparison
mlp_default = MLPClassifier(random_state=42, early_stopping=True, validation_fraction=0.1)
mlp_default.fit(X_train_scaled, y_train_encoded)
y_pred_default = mlp_default.predict(X_test_scaled)
accuracy_default = accuracy_score(y_test_encoded, y_pred_default)

# Use best model from grid search
best_mlp_model = grid_search.best_estimator_
y_pred_tuned = best_mlp_model.predict(X_test_scaled)
accuracy_tuned = accuracy_score(y_test_encoded, y_pred_tuned)

improvement = accuracy_tuned - accuracy_default

print(f"   Default Parameters Accuracy: {accuracy_default:.4f}")
print(f"   Tuned Parameters Accuracy:   {accuracy_tuned:.4f}")
print(f"   Improvement:                 {improvement:+.4f}")

if improvement > 0:
    print(f"   ✅ Tuning improved accuracy by {improvement:.2%}")
else:
    print(f"   ⚠️  Tuning did not improve accuracy")

# STEP 5: 10-Fold Cross-Validation with Tuned Model (for proper comparison)
print("\n" + "="*80)
print("🎯 10-FOLD CROSS-VALIDATION WITH TUNED PARAMETERS")
print("="*80)

print("🚀 Performing 10-fold cross-validation with tuned parameters...")

# Get cross-validated predictions for training set using tuned parameters
cv_predictions = cross_val_predict(best_mlp_model, X_train_scaled, y_train_encoded,
                                 cv=cv_strategy, method='predict')
cv_probabilities = cross_val_predict(best_mlp_model, X_train_scaled, y_train_encoded,
                                   cv=cv_strategy, method='predict_proba')

# Calculate training metrics as average across 10-fold CV
train_accuracy = accuracy_score(y_train_encoded, cv_predictions)

# Calculate metrics for BOTH CLASSES
train_precision = precision_score(y_train_encoded, cv_predictions, average=None)
train_recall = recall_score(y_train_encoded, cv_predictions, average=None)
train_f1 = f1_score(y_train_encoded, cv_predictions, average=None)
train_roc_auc = roc_auc_score(y_train_encoded, cv_probabilities[:, 1])
train_mcc = matthews_corrcoef(y_train_encoded, cv_predictions)

# Also get individual fold accuracies for reporting
cv_scores = cross_val_score(best_mlp_model, X_train_scaled, y_train_encoded,
                          cv=cv_strategy, scoring='accuracy', n_jobs=-1)

print("✅ 10-fold cross-validation completed!")

print(f"\n📊 10-FOLD CROSS-VALIDATION RESULTS (TRAINING PERFORMANCE):")
print(f"   Fold scores: {[f'{score:.4f}' for score in cv_scores]}")
print(f"   Mean CV Accuracy: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")
print(f"   Standard Deviation: {cv_scores.std():.4f}")

# STEP 6: Make predictions on TEST set with tuned model
print("\n" + "="*80)
print("📊 MODEL EVALUATION WITH TUNED PARAMETERS")
print("="*80)

# Predictions on TEST set with tuned model
y_test_pred = best_mlp_model.predict(X_test_scaled)
y_test_proba = best_mlp_model.predict_proba(X_test_scaled)

# Calculate metrics for TESTING set for BOTH CLASSES
test_accuracy = accuracy_score(y_test_encoded, y_test_pred)
test_precision = precision_score(y_test_encoded, y_test_pred, average=None)
test_recall = recall_score(y_test_encoded, y_test_pred, average=None)
test_f1 = f1_score(y_test_encoded, y_test_pred, average=None)
test_roc_auc = roc_auc_score(y_test_encoded, y_test_proba[:, 1])
test_mcc = matthews_corrcoef(y_test_encoded, y_test_pred)

# Confusion matrices
train_cm = confusion_matrix(y_train_encoded, cv_predictions)
test_cm = confusion_matrix(y_test_encoded, y_test_pred)
tn, fp, fn, tp = test_cm.ravel()

# Additional metrics
specificity = tn / (tn + fp)
npv = tn / (tn + fn)

# Performance gap
accuracy_gap = train_accuracy - test_accuracy

# Compute ROC curves for later saving
fpr_train, tpr_train, _ = roc_curve(y_train_encoded, cv_probabilities[:, 1])
fpr_test, tpr_test, _ = roc_curve(y_test_encoded, y_test_proba[:, 1])

# STEP 7: Display comprehensive results for BOTH CLASSES
print("\n" + "="*80)
print("📈 COMPREHENSIVE PERFORMANCE METRICS FOR BOTH CLASSES (TUNED MODEL)")
print("="*80)

# Create detailed comparison dataframe for BOTH CLASSES
class_performance_data = []

for i, class_name in enumerate(class_names):
    class_performance_data.extend([
        {
            'Class': class_name,
            'Metric': 'Precision',
            'Training (10-Fold CV)': f"{train_precision[i]:.4f}",
            'Testing': f"{test_precision[i]:.4f}",
            'Difference': f"{train_precision[i]-test_precision[i]:+.4f}"
        },
        {
            'Class': class_name,
            'Metric': 'Recall',
            'Training (10-Fold CV)': f"{train_recall[i]:.4f}",
            'Testing': f"{test_recall[i]:.4f}",
            'Difference': f"{train_recall[i]-test_recall[i]:+.4f}"
        },
        {
            'Class': class_name,
            'Metric': 'F1-Score',
            'Training (10-Fold CV)': f"{train_f1[i]:.4f}",
            'Testing': f"{test_f1[i]:.4f}",
            'Difference': f"{train_f1[i]-test_f1[i]:+.4f}"
        }
    ])

# Add overall metrics
overall_metrics = [
    {
        'Class': 'Overall',
        'Metric': 'Accuracy',
        'Training (10-Fold CV)': f"{train_accuracy:.4f}",
        'Testing': f"{test_accuracy:.4f}",
        'Difference': f"{train_accuracy-test_accuracy:+.4f}"
    },
    {
        'Class': 'Overall',
        'Metric': 'ROC-AUC',
        'Training (10-Fold CV)': f"{train_roc_auc:.4f}",
        'Testing': f"{test_roc_auc:.4f}",
        'Difference': f"{train_roc_auc-test_roc_auc:+.4f}"
    },
    {
        'Class': 'Overall',
        'Metric': 'MCC',
        'Training (10-Fold CV)': f"{train_mcc:.4f}",
        'Testing': f"{test_mcc:.4f}",
        'Difference': f"{train_mcc-test_mcc:+.4f}"
    }
]

performance_df = pd.DataFrame(class_performance_data + overall_metrics)
print(performance_df.to_string(index=False))

# STEP 8: Enhanced Performance Visualization for Both Classes (WITH ROC CURVE)
print("\n" + "="*80)
print("📊 GENERATING ENHANCED PERFORMANCE VISUALIZATIONS")
print("="*80)

# Create comprehensive performance comparison for both classes
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(20, 16))

# 1. Main Performance Comparison (Overall Metrics)
overall_metrics = ['Accuracy', 'ROC-AUC', 'MCC']
train_overall = [train_accuracy, train_roc_auc, train_mcc]
test_overall = [test_accuracy, test_roc_auc, test_mcc]

x = np.arange(len(overall_metrics))
width = 0.35

bars1 = ax1.bar(x - width/2, train_overall, width, label='Training (10-Fold CV)',
               color=elsevier_colors[0], alpha=0.8, edgecolor='black', linewidth=1.5)
bars2 = ax1.bar(x + width/2, test_overall, width, label='Testing (Held-Out)',
               color=elsevier_colors[1], alpha=0.8, edgecolor='black', linewidth=1.5)

# Add value labels on bars
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                f'{height:.3f}', ha='center', va='bottom',
                fontweight='bold', fontsize=11)

ax1.set_xlabel('Performance Metrics', fontsize=14, fontweight='bold')
ax1.set_ylabel('Score', fontsize=14, fontweight='bold')
ax1.set_title('MLP Classifier (Tuned) - Overall Performance\nTraining vs Testing',
              fontsize=16, fontweight='bold', pad=20)
ax1.set_xticks(x)
ax1.set_xticklabels(overall_metrics)
ax1.legend(fontsize=12, framealpha=0.9)
ax1.set_ylim(0, 1.1)
ax1.grid(True, alpha=0.3, linestyle='--')

# 2. ROC CURVE for Training and Testing Sets
ax2.plot(fpr_train, tpr_train, color=elsevier_colors[0], lw=2.5,
         label=f'Training ROC (AUC = {train_roc_auc:.3f})', alpha=0.8)
ax2.plot(fpr_test, tpr_test, color=elsevier_colors[1], lw=2.5,
         label=f'Testing ROC (AUC = {test_roc_auc:.3f})', alpha=0.8)

# Plot diagonal reference line
ax2.plot([0, 1], [0, 1], color='gray', lw=1.5, linestyle='--', alpha=0.7,
         label='Random Classifier (AUC = 0.5)')

# Fill area under curves
ax2.fill_between(fpr_train, tpr_train, alpha=0.2, color=elsevier_colors[0])
ax2.fill_between(fpr_test, tpr_test, alpha=0.2, color=elsevier_colors[1])

ax2.set_xlabel('False Positive Rate', fontsize=14, fontweight='bold')
ax2.set_ylabel('True Positive Rate', fontsize=14, fontweight='bold')
ax2.set_title('ROC Curves - Training vs Testing Sets\nMLP Classifier (Tuned)',
              fontsize=16, fontweight='bold', pad=20)
ax2.legend(loc='lower right', fontsize=12, framealpha=0.9)
ax2.grid(True, alpha=0.3, linestyle='--')
ax2.set_xlim([0.0, 1.0])
ax2.set_ylim([0.0, 1.05])

# 3. Confusion Matrix - Training
im1 = ax3.imshow(train_cm, cmap='Blues', interpolation='nearest', alpha=0.8)
ax3.set_xlabel('Predicted Label', fontsize=14, fontweight='bold')
ax3.set_ylabel('True Label', fontsize=14, fontweight='bold')
ax3.set_title('Training Set - Confusion Matrix\n(10-Fold CV Average)',
              fontsize=16, fontweight='bold', pad=20)
ax3.set_xticks([0, 1])
ax3.set_yticks([0, 1])
ax3.set_xticklabels(class_names)
ax3.set_yticklabels(class_names)

# Add text annotations for training CM
for i in range(train_cm.shape[0]):
    for j in range(train_cm.shape[1]):
        ax3.text(j, i, f'{train_cm[i, j]}\n({train_cm[i, j]/np.sum(train_cm):.1%})',
                ha='center', va='center', fontsize=12, fontweight='bold',
                color='white' if train_cm[i, j] > np.max(train_cm)/2 else 'black')

# 4. Confusion Matrix - Testing
im2 = ax4.imshow(test_cm, cmap='Reds', interpolation='nearest', alpha=0.8)
ax4.set_xlabel('Predicted Label', fontsize=14, fontweight='bold')
ax4.set_ylabel('True Label', fontsize=14, fontweight='bold')
ax4.set_title('Testing Set - Confusion Matrix\n(Held-Out Dataset)',
              fontsize=16, fontweight='bold', pad=20)
ax4.set_xticks([0, 1])
ax4.set_yticks([0, 1])
ax4.set_xticklabels(class_names)
ax4.set_yticklabels(class_names)

# Add text annotations for testing CM
for i in range(test_cm.shape[0]):
    for j in range(test_cm.shape[1]):
        ax4.text(j, i, f'{test_cm[i, j]}\n({test_cm[i, j]/np.sum(test_cm):.1%})',
                ha='center', va='center', fontsize=12, fontweight='bold',
                color='white' if test_cm[i, j] > np.max(test_cm)/2 else 'black')

plt.tight_layout()
plt.savefig('mlp_tuned_comprehensive_performance.png', dpi=300,
            bbox_inches='tight', facecolor='white')
plt.show()

# STEP 9: FIXED Classification Reports
print("\n" + "="*80)
print("📋 DETAILED CLASSIFICATION REPORTS FOR BOTH CLASSES")
print("="*80)

print("\nTRAINING SET CLASSIFICATION REPORT (10-Fold CV Average):")
training_report = classification_report(y_train_encoded, cv_predictions,
                                      target_names=class_names, digits=4)
print(training_report)

print("\nTESTING SET CLASSIFICATION REPORT (Held-Out):")
testing_report = classification_report(y_test_encoded, y_test_pred,
                                     target_names=class_names, digits=4)
print(testing_report)

# STEP 10: Individual Class Performance Analysis
print("\n" + "="*80)
print("🎯 INDIVIDUAL CLASS PERFORMANCE ANALYSIS")
print("="*80)

for i, class_name in enumerate(class_names):
    print(f"\n{class_name.upper()} CLASS PERFORMANCE:")
    print(f"  Training (10-Fold CV):")
    print(f"    Precision: {train_precision[i]:.4f}")
    print(f"    Recall:    {train_recall[i]:.4f}")
    print(f"    F1-Score:  {train_f1[i]:.4f}")
    print(f"  Testing (Held-Out):")
    print(f"    Precision: {test_precision[i]:.4f}")
    print(f"    Recall:    {test_recall[i]:.4f}")
    print(f"    F1-Score:  {test_f1[i]:.4f}")
    print(f"  Performance Gap:")
    print(f"    Precision: {train_precision[i]-test_precision[i]:+.4f}")
    print(f"    Recall:    {train_recall[i]-test_recall[i]:+.4f}")
    print(f"    F1-Score:  {train_f1[i]-test_f1[i]:+.4f}")

# STEP 11: Tuning Results Summary
print("\n" + "="*80)
print("🔬 HYPERPARAMETER TUNING SUMMARY")
print("="*80)

print(f"🎯 BEST PARAMETERS FOUND:")
for param, value in best_params.items():
    print(f"   {param}: {value}")

print(f"\n📊 PERFORMANCE COMPARISON:")
print(f"   Default Model Accuracy: {accuracy_default:.4f}")
print(f"   Tuned Model Accuracy:   {accuracy_tuned:.4f}")
print(f"   Improvement:            {improvement:+.4f}")

print(f"\n🚀 FINAL TUNED MODEL PERFORMANCE:")
print(f"   Training CV Accuracy: {train_accuracy:.4f}")
print(f"   Test Accuracy:        {test_accuracy:.4f}")
print(f"   Test ROC-AUC:         {test_roc_auc:.4f}")
print(f"   Test MCC:             {test_mcc:.4f}")

# STEP 12: Save ROC Data for Later Use (LIKE NAIVE BAYES AND DECISION TREE)
print("\n" + "="*80)
print("💾 SAVING ROC DATA FOR COMBINED VISUALIZATIONS")
print("="*80)

# Save the tuned model and scaler
joblib.dump(best_mlp_model, 'mlp_tuned_model.pkl')
joblib.dump(scaler, 'mlp_tuned_scaler.pkl')
joblib.dump(label_encoder, 'mlp_tuned_label_encoder.pkl')

# ============== SAVE ROC PREDICTION DATA ==============
print("\n💾 SAVING ROC PREDICTION DATA FILES...")

# File 1: Save NPZ file for ROC plotting
np.savez('mlp_roc_predictions.npz',
         # Training set data (from cross-validation)
         y_train_true=y_train_encoded,
         y_train_prob=cv_probabilities[:, 1],  # Probability for positive class

         # Testing set data
         y_test_true=y_test_encoded,
         y_test_prob=y_test_proba[:, 1],  # Probability for positive class

         # Precomputed ROC curve points
         fpr_train=fpr_train,
         tpr_train=tpr_train,
         fpr_test=fpr_test,
         tpr_test=tpr_test,

         # AUC values
         train_auc=train_roc_auc,
         test_auc=test_roc_auc,

         # Metadata
         model_name='MLP Classifier',
         class_names=class_names,
         feature_count=len(feature_columns),
         best_params=str(best_params),
         scaling_used=True)
print("✅ Saved: mlp_roc_predictions.npz")

# File 2: Save detailed predictions CSV
predictions_df = pd.DataFrame({
    'true_label': y_test_encoded,
    'predicted_label': y_test_pred,
    'probability_class_0': y_test_proba[:, 0],
    'probability_class_1': y_test_proba[:, 1],
    'correct': (y_test_encoded == y_test_pred)
})
predictions_df.to_csv('mlp_predictions.csv', index=False)
print("✅ Saved: mlp_predictions.csv")

# File 3: Save comprehensive metrics CSV
metrics_df = pd.DataFrame({
    'Model': ['MLP Classifier'],
    'Train_Accuracy': [train_accuracy],
    'Test_Accuracy': [test_accuracy],
    'Train_AUC': [train_roc_auc],
    'Test_AUC': [test_roc_auc],
    'Train_Precision_0': [train_precision[0]],
    'Train_Precision_1': [train_precision[1]],
    'Test_Precision_0': [test_precision[0]],
    'Test_Precision_1': [test_precision[1]],
    'Train_Recall_0': [train_recall[0]],
    'Train_Recall_1': [train_recall[1]],
    'Test_Recall_0': [test_recall[0]],
    'Test_Recall_1': [test_recall[1]],
    'Train_F1_0': [train_f1[0]],
    'Train_F1_1': [train_f1[1]],
    'Test_F1_0': [test_f1[0]],
    'Test_F1_1': [test_f1[1]],
    'Train_MCC': [train_mcc],
    'Test_MCC': [test_mcc],
    'Num_Features': [len(feature_columns)],
    'Hidden_Layers': [str(best_params.get('hidden_layer_sizes', 'N/A'))],
    'Activation': [best_params.get('activation', 'N/A')],
    'Learning_Rate': [best_params.get('learning_rate', 'N/A')],
    'Scaling_Used': ['Yes']
})
metrics_df.to_csv('mlp_metrics.csv', index=False)
print("✅ Saved: mlp_metrics.csv")

# File 4: Save all predictions (training + testing) for reference
all_predictions_df = pd.DataFrame({
    'dataset': ['train'] * len(y_train_encoded) + ['test'] * len(y_test_encoded),
    'true_label': np.concatenate([y_train_encoded, y_test_encoded]),
    'predicted_label': np.concatenate([cv_predictions, y_test_pred]),
    'probability_class_1': np.concatenate([cv_probabilities[:, 1], y_test_proba[:, 1]])
})
all_predictions_df.to_csv('mlp_all_predictions.csv', index=False)
print("✅ Saved: mlp_all_predictions.csv")

# File 5: Save grid search results
grid_results_df = pd.DataFrame(grid_search.cv_results_)
grid_results_df.to_excel('mlp_grid_search_results.xlsx', index=False)
print("✅ Saved: mlp_grid_search_results.xlsx")

# File 6: Save tuning parameters
tuning_params = {
    'best_params': best_params,
    'best_cv_score': grid_search.best_score_,
    'default_accuracy': accuracy_default,
    'tuned_accuracy': accuracy_tuned,
    'improvement': improvement,
    'test_roc_auc': test_roc_auc,
    'test_mcc': test_mcc
}
tuning_params_df = pd.DataFrame([tuning_params])
tuning_params_df.to_excel('mlp_tuning_parameters.xlsx', index=False)

print(f"\n🎯 MLP CLASSIFIER WITH HYPERPARAMETER TUNING COMPLETED!")
print(f"📊 Training (10-Fold CV) Accuracy: {train_accuracy:.1%}")
print(f"📊 Testing Accuracy: {test_accuracy:.1%}")
print(f"📈 Training ROC-AUC: {train_roc_auc:.3f}")
print(f"📈 Testing ROC-AUC: {test_roc_auc:.3f}")
print(f"⚡ Improvement over default: {improvement:+.2%}")
print(f"🔍 Performance metrics shown for BOTH classes: {class_names}")

print(f"\n📁 FILES GENERATED FOR ROC CURVES:")
print(f"   1. mlp_roc_predictions.npz     - Main ROC data file")
print(f"   2. mlp_predictions.csv         - Detailed test predictions")
print(f"   3. mlp_metrics.csv             - Performance metrics")
print(f"   4. mlp_all_predictions.csv     - All predictions (train+test)")
print(f"   5. mlp_tuned_model.pkl         - Trained model")
print(f"   6. mlp_tuned_scaler.pkl        - Scaler for new data")
print(f"   7. mlp_grid_search_results.xlsx - Grid search results")

print(f"\n🎯 Use 'mlp_roc_predictions.npz' with other model files for combined ROC plots!")
print("🚀 TUNED MLP MODEL READY FOR DEPLOYMENT!")

### **Decission Tree:**

Requirements: training-set-selected-features.xlsx, testing-set-selected-features.xlsx

In [ ]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                           roc_auc_score, roc_curve, matthews_corrcoef, confusion_matrix,
                           classification_report)
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score, StratifiedKFold, cross_val_predict, GridSearchCV
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

# Set publication-quality parameters (Fixed font settings - REMOVED PROBLEMATIC FONTS)
plt.rcParams.update({
    'font.size': 12,
    'axes.labelsize': 14,
    'axes.titlesize': 16,
    'xtick.labelsize': 12,
    'ytick.labelsize': 12,
    'legend.fontsize': 12,
    'figure.titlesize': 18,
    'font.family': ['DejaVu Sans', 'Liberation Sans', 'Bitstream Vera Sans', 'sans-serif']
})

# Professional Elsevier color scheme
elsevier_colors = ['#2E86AB', '#A23B72', '#F18F01', '#C73E1D', '#1A936F', '#114B5F']

print("🌳 DECISION TREE CLASSIFIER WITH HYPERPARAMETER TUNING (10-FOLD CV + GRID SEARCH)")
print("="*80)

# STEP 1: Load the feature-selected datasets
print("📊 Loading feature-selected datasets...")

train_df = pd.read_excel('training_set_selected_features.xlsx')
test_df = pd.read_excel('testing_set_selected_features.xlsx')

print(f"Training set: {train_df.shape}")
print(f"Testing set: {test_df.shape}")

# STEP 2: Prepare features and target
print("\n" + "="*80)
print("🔧 PREPARING DATA")
print("="*80)

# Identify feature columns (exclude metadata)
metadata_columns = ['COMPOUND ID', 'SMILE CODE', 'Ligand Type']
feature_columns = [col for col in train_df.columns if col not in metadata_columns]

X_train = train_df[feature_columns]
y_train = train_df['Ligand Type']

X_test = test_df[feature_columns]
y_test = test_df['Ligand Type']

print(f"Selected features: {len(feature_columns)}")
print(f"Training samples: {X_train.shape[0]}")
print(f"Testing samples: {X_test.shape[0]}")

# Encode labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# FIXED: Properly get class names as strings
class_names = [str(cls) for cls in label_encoder.classes_]
print(f"Class names: {class_names}")

print("✅ Data prepared successfully!")

# STEP 3: Hyperparameter Tuning with GridSearchCV and 10-Fold CV
print("\n" + "="*80)
print("🎯 HYPERPARAMETER TUNING WITH GRIDSEARCHCV (10-FOLD CV)")
print("="*80)

# Define parameter grid for Decision Tree
param_grid = {
    'max_depth': [3, 5, 7, 10, 15, None],
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 2, 5, 10],
    'criterion': ['gini', 'entropy'],
    'max_features': ['sqrt', 'log2', None]
}

# Initialize Decision Tree
dt = DecisionTreeClassifier(random_state=42)

# 10-fold cross-validation strategy
cv_strategy = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# GridSearchCV with 10-fold CV
print("🔍 Performing Grid Search with 10-fold CV...")
print(f"   Parameter grid: {param_grid}")
print(f"   Total parameter combinations: {len(param_grid['max_depth']) * len(param_grid['min_samples_split']) * len(param_grid['min_samples_leaf']) * len(param_grid['criterion']) * len(param_grid['max_features'])}")
print(f"   This may take a while...")

grid_search = GridSearchCV(
    estimator=dt,
    param_grid=param_grid,
    cv=cv_strategy,
    scoring='accuracy',
    n_jobs=-1,
    verbose=1
)

# Perform grid search
grid_search.fit(X_train, y_train_encoded)

print("✅ Grid Search completed!")

# Display best parameters and scores
print(f"\n🎯 BEST PARAMETERS FOUND:")
best_params = grid_search.best_params_
for param, value in best_params.items():
    print(f"   {param}: {value}")
print(f"   Best CV Score: {grid_search.best_score_:.4f}")

# STEP 4: Compare Default vs Tuned Models
print("\n" + "="*80)
print("📊 COMPARISON: DEFAULT vs TUNED PARAMETERS")
print("="*80)

# Train model with default parameters for comparison
dt_default = DecisionTreeClassifier(random_state=42)
dt_default.fit(X_train, y_train_encoded)
y_pred_default = dt_default.predict(X_test)
accuracy_default = accuracy_score(y_test_encoded, y_pred_default)

# Use best model from grid search
best_dt_model = grid_search.best_estimator_
y_pred_tuned = best_dt_model.predict(X_test)
accuracy_tuned = accuracy_score(y_test_encoded, y_pred_tuned)

improvement = accuracy_tuned - accuracy_default

print(f"   Default Parameters Accuracy: {accuracy_default:.4f}")
print(f"   Tuned Parameters Accuracy:   {accuracy_tuned:.4f}")
print(f"   Improvement:                 {improvement:+.4f}")

if improvement > 0:
    print(f"   ✅ Tuning improved accuracy by {improvement:.2%}")
else:
    print(f"   ⚠️  Tuning did not improve accuracy")

# STEP 5: 10-Fold Cross-Validation with Tuned Model (for proper comparison)
print("\n" + "="*80)
print("🎯 10-FOLD CROSS-VALIDATION WITH TUNED PARAMETERS")
print("="*80)

print("🚀 Performing 10-fold cross-validation with tuned parameters...")

# Get cross-validated predictions for training set using tuned parameters
cv_predictions = cross_val_predict(best_dt_model, X_train, y_train_encoded,
                                 cv=cv_strategy, method='predict')
cv_probabilities = cross_val_predict(best_dt_model, X_train, y_train_encoded,
                                   cv=cv_strategy, method='predict_proba')

# Calculate training metrics as average across 10-fold CV
train_accuracy = accuracy_score(y_train_encoded, cv_predictions)

# Calculate metrics for BOTH CLASSES
train_precision = precision_score(y_train_encoded, cv_predictions, average=None)
train_recall = recall_score(y_train_encoded, cv_predictions, average=None)
train_f1 = f1_score(y_train_encoded, cv_predictions, average=None)
train_roc_auc = roc_auc_score(y_train_encoded, cv_probabilities[:, 1])
train_mcc = matthews_corrcoef(y_train_encoded, cv_predictions)

# Also get individual fold accuracies for reporting
cv_scores = cross_val_score(best_dt_model, X_train, y_train_encoded,
                          cv=cv_strategy, scoring='accuracy', n_jobs=-1)

print("✅ 10-fold cross-validation completed!")

print(f"\n📊 10-FOLD CROSS-VALIDATION RESULTS (TRAINING PERFORMANCE):")
print(f"   Fold scores: {[f'{score:.4f}' for score in cv_scores]}")
print(f"   Mean CV Accuracy: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")
print(f"   Standard Deviation: {cv_scores.std():.4f}")

# STEP 6: Make predictions on TEST set with tuned model
print("\n" + "="*80)
print("📊 MODEL EVALUATION WITH TUNED PARAMETERS")
print("="*80)

# Predictions on TEST set with tuned model
y_test_pred = best_dt_model.predict(X_test)
y_test_proba = best_dt_model.predict_proba(X_test)

# Calculate metrics for TESTING set for BOTH CLASSES
test_accuracy = accuracy_score(y_test_encoded, y_test_pred)
test_precision = precision_score(y_test_encoded, y_test_pred, average=None)
test_recall = recall_score(y_test_encoded, y_test_pred, average=None)
test_f1 = f1_score(y_test_encoded, y_test_pred, average=None)
test_roc_auc = roc_auc_score(y_test_encoded, y_test_proba[:, 1])
test_mcc = matthews_corrcoef(y_test_encoded, y_test_pred)

# Confusion matrices
train_cm = confusion_matrix(y_train_encoded, cv_predictions)
test_cm = confusion_matrix(y_test_encoded, y_test_pred)
tn, fp, fn, tp = test_cm.ravel()

# Additional metrics
specificity = tn / (tn + fp)
npv = tn / (tn + fn)

# Performance gap
accuracy_gap = train_accuracy - test_accuracy

# Compute ROC curves for later saving
fpr_train, tpr_train, _ = roc_curve(y_train_encoded, cv_probabilities[:, 1])
fpr_test, tpr_test, _ = roc_curve(y_test_encoded, y_test_proba[:, 1])

# STEP 7: Display comprehensive results for BOTH CLASSES
print("\n" + "="*80)
print("📈 COMPREHENSIVE PERFORMANCE METRICS FOR BOTH CLASSES (TUNED MODEL)")
print("="*80)

# Create detailed comparison dataframe for BOTH CLASSES
class_performance_data = []

for i, class_name in enumerate(class_names):
    class_performance_data.extend([
        {
            'Class': class_name,
            'Metric': 'Precision',
            'Training (10-Fold CV)': f"{train_precision[i]:.4f}",
            'Testing': f"{test_precision[i]:.4f}",
            'Difference': f"{train_precision[i]-test_precision[i]:+.4f}"
        },
        {
            'Class': class_name,
            'Metric': 'Recall',
            'Training (10-Fold CV)': f"{train_recall[i]:.4f}",
            'Testing': f"{test_recall[i]:.4f}",
            'Difference': f"{train_recall[i]-test_recall[i]:+.4f}"
        },
        {
            'Class': class_name,
            'Metric': 'F1-Score',
            'Training (10-Fold CV)': f"{train_f1[i]:.4f}",
            'Testing': f"{test_f1[i]:.4f}",
            'Difference': f"{train_f1[i]-test_f1[i]:+.4f}"
        }
    ])

# Add overall metrics
overall_metrics = [
    {
        'Class': 'Overall',
        'Metric': 'Accuracy',
        'Training (10-Fold CV)': f"{train_accuracy:.4f}",
        'Testing': f"{test_accuracy:.4f}",
        'Difference': f"{train_accuracy-test_accuracy:+.4f}"
    },
    {
        'Class': 'Overall',
        'Metric': 'ROC-AUC',
        'Training (10-Fold CV)': f"{train_roc_auc:.4f}",
        'Testing': f"{test_roc_auc:.4f}",
        'Difference': f"{train_roc_auc-test_roc_auc:+.4f}"
    },
    {
        'Class': 'Overall',
        'Metric': 'MCC',
        'Training (10-Fold CV)': f"{train_mcc:.4f}",
        'Testing': f"{test_mcc:.4f}",
        'Difference': f"{train_mcc-test_mcc:+.4f}"
    }
]

performance_df = pd.DataFrame(class_performance_data + overall_metrics)
print(performance_df.to_string(index=False))

# STEP 8: Enhanced Performance Visualization for Both Classes (WITH ROC CURVE)
print("\n" + "="*80)
print("📊 GENERATING ENHANCED PERFORMANCE VISUALIZATIONS")
print("="*80)

# Create comprehensive performance comparison for both classes
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(20, 16))

# 1. Main Performance Comparison (Overall Metrics)
overall_metrics = ['Accuracy', 'ROC-AUC', 'MCC']
train_overall = [train_accuracy, train_roc_auc, train_mcc]
test_overall = [test_accuracy, test_roc_auc, test_mcc]

x = np.arange(len(overall_metrics))
width = 0.35

bars1 = ax1.bar(x - width/2, train_overall, width, label='Training (10-Fold CV)',
               color=elsevier_colors[0], alpha=0.8, edgecolor='black', linewidth=1.5)
bars2 = ax1.bar(x + width/2, test_overall, width, label='Testing (Held-Out)',
               color=elsevier_colors[1], alpha=0.8, edgecolor='black', linewidth=1.5)

# Add value labels on bars
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                f'{height:.3f}', ha='center', va='bottom',
                fontweight='bold', fontsize=11)

ax1.set_xlabel('Performance Metrics', fontsize=14, fontweight='bold')
ax1.set_ylabel('Score', fontsize=14, fontweight='bold')
ax1.set_title('Decision Tree (Tuned) - Overall Performance\nTraining vs Testing',
              fontsize=16, fontweight='bold', pad=20)
ax1.set_xticks(x)
ax1.set_xticklabels(overall_metrics)
ax1.legend(fontsize=12, framealpha=0.9)
ax1.set_ylim(0, 1.1)
ax1.grid(True, alpha=0.3, linestyle='--')

# 2. ROC CURVE for Training and Testing Sets
ax2.plot(fpr_train, tpr_train, color=elsevier_colors[0], lw=2.5,
         label=f'Training ROC (AUC = {train_roc_auc:.3f})', alpha=0.8)
ax2.plot(fpr_test, tpr_test, color=elsevier_colors[1], lw=2.5,
         label=f'Testing ROC (AUC = {test_roc_auc:.3f})', alpha=0.8)

# Plot diagonal reference line
ax2.plot([0, 1], [0, 1], color='gray', lw=1.5, linestyle='--', alpha=0.7,
         label='Random Classifier (AUC = 0.5)')

# Fill area under curves
ax2.fill_between(fpr_train, tpr_train, alpha=0.2, color=elsevier_colors[0])
ax2.fill_between(fpr_test, tpr_test, alpha=0.2, color=elsevier_colors[1])

ax2.set_xlabel('False Positive Rate', fontsize=14, fontweight='bold')
ax2.set_ylabel('True Positive Rate', fontsize=14, fontweight='bold')
ax2.set_title('ROC Curves - Training vs Testing Sets\nDecision Tree (Tuned)',
              fontsize=16, fontweight='bold', pad=20)
ax2.legend(loc='lower right', fontsize=12, framealpha=0.9)
ax2.grid(True, alpha=0.3, linestyle='--')
ax2.set_xlim([0.0, 1.0])
ax2.set_ylim([0.0, 1.05])

# 3. Confusion Matrix - Training
im1 = ax3.imshow(train_cm, cmap='Blues', interpolation='nearest', alpha=0.8)
ax3.set_xlabel('Predicted Label', fontsize=14, fontweight='bold')
ax3.set_ylabel('True Label', fontsize=14, fontweight='bold')
ax3.set_title('Training Set - Confusion Matrix\n(10-Fold CV Average)',
              fontsize=16, fontweight='bold', pad=20)
ax3.set_xticks([0, 1])
ax3.set_yticks([0, 1])
ax3.set_xticklabels(class_names)
ax3.set_yticklabels(class_names)

# Add text annotations for training CM
for i in range(train_cm.shape[0]):
    for j in range(train_cm.shape[1]):
        ax3.text(j, i, f'{train_cm[i, j]}\n({train_cm[i, j]/np.sum(train_cm):.1%})',
                ha='center', va='center', fontsize=12, fontweight='bold',
                color='white' if train_cm[i, j] > np.max(train_cm)/2 else 'black')

# 4. Confusion Matrix - Testing
im2 = ax4.imshow(test_cm, cmap='Reds', interpolation='nearest', alpha=0.8)
ax4.set_xlabel('Predicted Label', fontsize=14, fontweight='bold')
ax4.set_ylabel('True Label', fontsize=14, fontweight='bold')
ax4.set_title('Testing Set - Confusion Matrix\n(Held-Out Dataset)',
              fontsize=16, fontweight='bold', pad=20)
ax4.set_xticks([0, 1])
ax4.set_yticks([0, 1])
ax4.set_xticklabels(class_names)
ax4.set_yticklabels(class_names)

# Add text annotations for testing CM
for i in range(test_cm.shape[0]):
    for j in range(test_cm.shape[1]):
        ax4.text(j, i, f'{test_cm[i, j]}\n({test_cm[i, j]/np.sum(test_cm):.1%})',
                ha='center', va='center', fontsize=12, fontweight='bold',
                color='white' if test_cm[i, j] > np.max(test_cm)/2 else 'black')

plt.tight_layout()
plt.savefig('decision_tree_tuned_comprehensive_performance.png', dpi=300,
            bbox_inches='tight', facecolor='white')
plt.show()

# STEP 9: FIXED Classification Reports
print("\n" + "="*80)
print("📋 DETAILED CLASSIFICATION REPORTS FOR BOTH CLASSES")
print("="*80)

print("\nTRAINING SET CLASSIFICATION REPORT (10-Fold CV Average):")
training_report = classification_report(y_train_encoded, cv_predictions,
                                      target_names=class_names, digits=4)
print(training_report)

print("\nTESTING SET CLASSIFICATION REPORT (Held-Out):")
testing_report = classification_report(y_test_encoded, y_test_pred,
                                     target_names=class_names, digits=4)
print(testing_report)

# STEP 10: Individual Class Performance Analysis
print("\n" + "="*80)
print("🎯 INDIVIDUAL CLASS PERFORMANCE ANALYSIS")
print("="*80)

for i, class_name in enumerate(class_names):
    print(f"\n{class_name.upper()} CLASS PERFORMANCE:")
    print(f"  Training (10-Fold CV):")
    print(f"    Precision: {train_precision[i]:.4f}")
    print(f"    Recall:    {train_recall[i]:.4f}")
    print(f"    F1-Score:  {train_f1[i]:.4f}")
    print(f"  Testing (Held-Out):")
    print(f"    Precision: {test_precision[i]:.4f}")
    print(f"    Recall:    {test_recall[i]:.4f}")
    print(f"    F1-Score:  {test_f1[i]:.4f}")
    print(f"  Performance Gap:")
    print(f"    Precision: {train_precision[i]-test_precision[i]:+.4f}")
    print(f"    Recall:    {train_recall[i]-test_recall[i]:+.4f}")
    print(f"    F1-Score:  {train_f1[i]-test_f1[i]:+.4f}")

# STEP 11: Tuning Results Summary
print("\n" + "="*80)
print("🔬 HYPERPARAMETER TUNING SUMMARY")
print("="*80)

print(f"🎯 BEST PARAMETERS FOUND:")
for param, value in best_params.items():
    print(f"   {param}: {value}")

print(f"\n📊 PERFORMANCE COMPARISON:")
print(f"   Default Model Accuracy: {accuracy_default:.4f}")
print(f"   Tuned Model Accuracy:   {accuracy_tuned:.4f}")
print(f"   Improvement:            {improvement:+.4f}")

print(f"\n🚀 FINAL TUNED MODEL PERFORMANCE:")
print(f"   Training CV Accuracy: {train_accuracy:.4f}")
print(f"   Test Accuracy:        {test_accuracy:.4f}")
print(f"   Test ROC-AUC:         {test_roc_auc:.4f}")
print(f"   Test MCC:             {test_mcc:.4f}")

# STEP 12: Feature Importance Analysis (Specific to Decision Trees)
print("\n" + "="*80)
print("🔍 FEATURE IMPORTANCE ANALYSIS")
print("="*80)

# Get feature importances from the tuned model
feature_importances = best_dt_model.feature_importances_

# Create a DataFrame for feature importance
importance_df = pd.DataFrame({
    'Feature': feature_columns,
    'Importance': feature_importances
})

# Sort by importance
importance_df = importance_df.sort_values('Importance', ascending=False)

# Display top 20 most important features
print("📊 TOP 20 MOST IMPORTANT FEATURES:")
print(importance_df.head(20).to_string(index=False))

# Plot feature importance
plt.figure(figsize=(12, 8))
top_features = importance_df.head(15)
plt.barh(top_features['Feature'], top_features['Importance'], color=elsevier_colors[0])
plt.xlabel('Feature Importance', fontsize=14, fontweight='bold')
plt.title('Decision Tree - Top 15 Feature Importances', fontsize=16, fontweight='bold')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.savefig('decision_tree_feature_importances.png', dpi=300, bbox_inches='tight', facecolor='white')
plt.show()

# STEP 13: Save ROC Data for Later Use (LIKE NAIVE BAYES)
print("\n" + "="*80)
print("💾 SAVING ROC DATA FOR COMBINED VISUALIZATIONS")
print("="*80)

# Save the tuned model
joblib.dump(best_dt_model, 'decision_tree_tuned_model.pkl')
joblib.dump(label_encoder, 'decision_tree_tuned_label_encoder.pkl')

# ============== SAVE ROC PREDICTION DATA ==============
print("\n💾 SAVING ROC PREDICTION DATA FILES...")

# File 1: Save NPZ file for ROC plotting
np.savez('decision_tree_roc_predictions.npz',
         # Training set data (from cross-validation)
         y_train_true=y_train_encoded,
         y_train_prob=cv_probabilities[:, 1],  # Probability for positive class

         # Testing set data
         y_test_true=y_test_encoded,
         y_test_prob=y_test_proba[:, 1],  # Probability for positive class

         # Precomputed ROC curve points
         fpr_train=fpr_train,
         tpr_train=tpr_train,
         fpr_test=fpr_test,
         tpr_test=tpr_test,

         # AUC values
         train_auc=train_roc_auc,
         test_auc=test_roc_auc,

         # Metadata
         model_name='Decision Tree',
         class_names=class_names,
         feature_count=len(feature_columns),
         best_params=str(best_params))
print("✅ Saved: decision_tree_roc_predictions.npz")

# File 2: Save detailed predictions CSV
predictions_df = pd.DataFrame({
    'true_label': y_test_encoded,
    'predicted_label': y_test_pred,
    'probability_class_0': y_test_proba[:, 0],
    'probability_class_1': y_test_proba[:, 1],
    'correct': (y_test_encoded == y_test_pred)
})
predictions_df.to_csv('decision_tree_predictions.csv', index=False)
print("✅ Saved: decision_tree_predictions.csv")

# File 3: Save comprehensive metrics CSV
metrics_df = pd.DataFrame({
    'Model': ['Decision Tree'],
    'Train_Accuracy': [train_accuracy],
    'Test_Accuracy': [test_accuracy],
    'Train_AUC': [train_roc_auc],
    'Test_AUC': [test_roc_auc],
    'Train_Precision_0': [train_precision[0]],
    'Train_Precision_1': [train_precision[1]],
    'Test_Precision_0': [test_precision[0]],
    'Test_Precision_1': [test_precision[1]],
    'Train_Recall_0': [train_recall[0]],
    'Train_Recall_1': [train_recall[1]],
    'Test_Recall_0': [test_recall[0]],
    'Test_Recall_1': [test_recall[1]],
    'Train_F1_0': [train_f1[0]],
    'Train_F1_1': [train_f1[1]],
    'Test_F1_0': [test_f1[0]],
    'Test_F1_1': [test_f1[1]],
    'Train_MCC': [train_mcc],
    'Test_MCC': [test_mcc],
    'Num_Features': [len(feature_columns)],
    'Max_Depth': [best_params.get('max_depth', 'N/A')],
    'Criterion': [best_params.get('criterion', 'N/A')]
})
metrics_df.to_csv('decision_tree_metrics.csv', index=False)
print("✅ Saved: decision_tree_metrics.csv")

# File 4: Save all predictions (training + testing) for reference
all_predictions_df = pd.DataFrame({
    'dataset': ['train'] * len(y_train_encoded) + ['test'] * len(y_test_encoded),
    'true_label': np.concatenate([y_train_encoded, y_test_encoded]),
    'predicted_label': np.concatenate([cv_predictions, y_test_pred]),
    'probability_class_1': np.concatenate([cv_probabilities[:, 1], y_test_proba[:, 1]])
})
all_predictions_df.to_csv('decision_tree_all_predictions.csv', index=False)
print("✅ Saved: decision_tree_all_predictions.csv")

# File 5: Save feature importances
importance_df.to_excel('decision_tree_feature_importances.xlsx', index=False)

# File 6: Save grid search results
grid_results_df = pd.DataFrame(grid_search.cv_results_)
grid_results_df.to_excel('decision_tree_grid_search_results.xlsx', index=False)
print("✅ Saved: decision_tree_grid_search_results.xlsx")

# File 7: Save tuning parameters
tuning_params = {
    'best_params': best_params,
    'best_cv_score': grid_search.best_score_,
    'default_accuracy': accuracy_default,
    'tuned_accuracy': accuracy_tuned,
    'improvement': improvement,
    'test_roc_auc': test_roc_auc,
    'test_mcc': test_mcc
}
tuning_params_df = pd.DataFrame([tuning_params])
tuning_params_df.to_excel('decision_tree_tuning_parameters.xlsx', index=False)

print(f"\n🎯 DECISION TREE WITH HYPERPARAMETER TUNING COMPLETED!")
print(f"📊 Training (10-Fold CV) Accuracy: {train_accuracy:.1%}")
print(f"📊 Testing Accuracy: {test_accuracy:.1%}")
print(f"📈 Training ROC-AUC: {train_roc_auc:.3f}")
print(f"📈 Testing ROC-AUC: {test_roc_auc:.3f}")
print(f"⚡ Improvement over default: {improvement:+.2%}")
print(f"🔍 Performance metrics shown for BOTH classes: {class_names}")

print(f"\n📁 FILES GENERATED FOR ROC CURVES:")
print(f"   1. decision_tree_roc_predictions.npz     - Main ROC data file")
print(f"   2. decision_tree_predictions.csv         - Detailed test predictions")
print(f"   3. decision_tree_metrics.csv             - Performance metrics")
print(f"   4. decision_tree_all_predictions.csv     - All predictions (train+test)")
print(f"   5. decision_tree_tuned_model.pkl         - Trained model")
print(f"   6. decision_tree_grid_search_results.xlsx - Grid search results")
print(f"   7. decision_tree_feature_importances.xlsx - Feature importance scores")

print(f"\n🎯 Use 'decision_tree_roc_predictions.npz' with Naive Bayes file for combined ROC plots!")
print("🌳 TUNED DECISION TREE READY FOR DEPLOYMENT!")

### **XGBoost Classifier:**

Requirements: training-set-selected-features.xlsx, testing-set-selected-features.xlsx

In [ ]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                           roc_auc_score, roc_curve, matthews_corrcoef, confusion_matrix,
                           classification_report)
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score, StratifiedKFold, cross_val_predict, GridSearchCV
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

# Set publication-quality parameters (Fixed font settings)
plt.rcParams['font.size'] = 12
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['axes.titlesize'] = 16
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
plt.rcParams['legend.fontsize'] = 12
plt.rcParams['figure.titlesize'] = 18

# Use available system fonts to avoid warnings
plt.rcParams['font.family'] = ['DejaVu Sans', 'Arial', 'Helvetica', 'sans-serif']

# Professional Elsevier color scheme
elsevier_colors = ['#2E86AB', '#A23B72', '#F18F01', '#C73E1D', '#1A936F', '#114B5F']

print("🚀 XGBOOST CLASSIFIER WITH HYPERPARAMETER TUNING (10-FOLD CV + GRID SEARCH)")
print("="*80)

# STEP 1: Load the feature-selected datasets
print("📊 Loading feature-selected datasets...")

train_df = pd.read_excel('training_set_selected_features.xlsx')
test_df = pd.read_excel('testing_set_selected_features.xlsx')

print(f"Training set: {train_df.shape}")
print(f"Testing set: {test_df.shape}")

# STEP 2: Prepare features and target
print("\n" + "="*80)
print("🔧 PREPARING DATA")
print("="*80)

# Identify feature columns (exclude metadata)
metadata_columns = ['COMPOUND ID', 'SMILE CODE', 'Ligand Type']
feature_columns = [col for col in train_df.columns if col not in metadata_columns]

X_train = train_df[feature_columns]
y_train = train_df['Ligand Type']

X_test = test_df[feature_columns]
y_test = test_df['Ligand Type']

print(f"Selected features: {len(feature_columns)}")
print(f"Training samples: {X_train.shape[0]}")
print(f"Testing samples: {X_test.shape[0]}")

# Encode labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# FIXED: Properly get class names as strings
class_names = [str(cls) for cls in label_encoder.classes_]
print(f"Class names: {class_names}")

print("✅ Data prepared successfully!")

# STEP 3: Hyperparameter Tuning with GridSearchCV and 10-Fold CV
print("\n" + "="*80)
print("🎯 HYPERPARAMETER TUNING WITH GRIDSEARCHCV (10-FOLD CV)")
print("="*80)

# Define parameter grid for XGBoost
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0],
    'gamma': [0, 0.1, 0.2]
}

# Initialize XGBoost
xgb = XGBClassifier(random_state=42, eval_metric='logloss')

# 10-fold cross-validation strategy
cv_strategy = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# GridSearchCV with 10-fold CV
print("🔍 Performing Grid Search with 10-fold CV...")
print(f"   Parameter grid: {param_grid}")
print(f"   Total parameter combinations: {len(param_grid['n_estimators']) * len(param_grid['max_depth']) * len(param_grid['learning_rate']) * len(param_grid['subsample']) * len(param_grid['colsample_bytree']) * len(param_grid['gamma'])}")
print(f"   This may take a while...")

grid_search = GridSearchCV(
    estimator=xgb,
    param_grid=param_grid,
    cv=cv_strategy,
    scoring='accuracy',
    n_jobs=-1,
    verbose=1
)

# Perform grid search
grid_search.fit(X_train, y_train_encoded)

print("✅ Grid Search completed!")

# Display best parameters and scores
print(f"\n🎯 BEST PARAMETERS FOUND:")
best_params = grid_search.best_params_
for param, value in best_params.items():
    print(f"   {param}: {value}")
print(f"   Best CV Score: {grid_search.best_score_:.4f}")

# STEP 4: Compare Default vs Tuned Models
print("\n" + "="*80)
print("📊 COMPARISON: DEFAULT vs TUNED PARAMETERS")
print("="*80)

# Train model with default parameters for comparison
xgb_default = XGBClassifier(random_state=42, eval_metric='logloss')
xgb_default.fit(X_train, y_train_encoded)
y_pred_default = xgb_default.predict(X_test)
accuracy_default = accuracy_score(y_test_encoded, y_pred_default)

# Use best model from grid search
best_xgb_model = grid_search.best_estimator_
y_pred_tuned = best_xgb_model.predict(X_test)
accuracy_tuned = accuracy_score(y_test_encoded, y_pred_tuned)

improvement = accuracy_tuned - accuracy_default

print(f"   Default Parameters Accuracy: {accuracy_default:.4f}")
print(f"   Tuned Parameters Accuracy:   {accuracy_tuned:.4f}")
print(f"   Improvement:                 {improvement:+.4f}")

if improvement > 0:
    print(f"   ✅ Tuning improved accuracy by {improvement:.2%}")
else:
    print(f"   ⚠️  Tuning did not improve accuracy")

# STEP 5: 10-Fold Cross-Validation with Tuned Model (for proper comparison)
print("\n" + "="*80)
print("🎯 10-FOLD CROSS-VALIDATION WITH TUNED PARAMETERS")
print("="*80)

print("🚀 Performing 10-fold cross-validation with tuned parameters...")

# Get cross-validated predictions for training set using tuned parameters
cv_predictions = cross_val_predict(best_xgb_model, X_train, y_train_encoded,
                                 cv=cv_strategy, method='predict')
cv_probabilities = cross_val_predict(best_xgb_model, X_train, y_train_encoded,
                                   cv=cv_strategy, method='predict_proba')

# Calculate training metrics as average across 10-fold CV
train_accuracy = accuracy_score(y_train_encoded, cv_predictions)

# Calculate metrics for BOTH CLASSES
train_precision = precision_score(y_train_encoded, cv_predictions, average=None)
train_recall = recall_score(y_train_encoded, cv_predictions, average=None)
train_f1 = f1_score(y_train_encoded, cv_predictions, average=None)
train_roc_auc = roc_auc_score(y_train_encoded, cv_probabilities[:, 1])
train_mcc = matthews_corrcoef(y_train_encoded, cv_predictions)

# Also get individual fold accuracies for reporting
cv_scores = cross_val_score(best_xgb_model, X_train, y_train_encoded,
                          cv=cv_strategy, scoring='accuracy', n_jobs=-1)

print("✅ 10-fold cross-validation completed!")

print(f"\n📊 10-FOLD CROSS-VALIDATION RESULTS (TRAINING PERFORMANCE):")
print(f"   Fold scores: {[f'{score:.4f}' for score in cv_scores]}")
print(f"   Mean CV Accuracy: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")
print(f"   Standard Deviation: {cv_scores.std():.4f}")

# STEP 6: Make predictions on TEST set with tuned model
print("\n" + "="*80)
print("📊 MODEL EVALUATION WITH TUNED PARAMETERS")
print("="*80)

# Predictions on TEST set with tuned model
y_test_pred = best_xgb_model.predict(X_test)
y_test_proba = best_xgb_model.predict_proba(X_test)

# Calculate metrics for TESTING set for BOTH CLASSES
test_accuracy = accuracy_score(y_test_encoded, y_test_pred)
test_precision = precision_score(y_test_encoded, y_test_pred, average=None)
test_recall = recall_score(y_test_encoded, y_test_pred, average=None)
test_f1 = f1_score(y_test_encoded, y_test_pred, average=None)
test_roc_auc = roc_auc_score(y_test_encoded, y_test_proba[:, 1])
test_mcc = matthews_corrcoef(y_test_encoded, y_test_pred)

# Confusion matrices
train_cm = confusion_matrix(y_train_encoded, cv_predictions)
test_cm = confusion_matrix(y_test_encoded, y_test_pred)
tn, fp, fn, tp = test_cm.ravel()

# Additional metrics
specificity = tn / (tn + fp)
npv = tn / (tn + fn)

# Performance gap
accuracy_gap = train_accuracy - test_accuracy

# Compute ROC curves for later saving
fpr_train, tpr_train, _ = roc_curve(y_train_encoded, cv_probabilities[:, 1])
fpr_test, tpr_test, _ = roc_curve(y_test_encoded, y_test_proba[:, 1])

# STEP 7: Display comprehensive results for BOTH CLASSES
print("\n" + "="*80)
print("📈 COMPREHENSIVE PERFORMANCE METRICS FOR BOTH CLASSES (TUNED MODEL)")
print("="*80)

# Create detailed comparison dataframe for BOTH CLASSES
class_performance_data = []

for i, class_name in enumerate(class_names):
    class_performance_data.extend([
        {
            'Class': class_name,
            'Metric': 'Precision',
            'Training (10-Fold CV)': f"{train_precision[i]:.4f}",
            'Testing': f"{test_precision[i]:.4f}",
            'Difference': f"{train_precision[i]-test_precision[i]:+.4f}"
        },
        {
            'Class': class_name,
            'Metric': 'Recall',
            'Training (10-Fold CV)': f"{train_recall[i]:.4f}",
            'Testing': f"{test_recall[i]:.4f}",
            'Difference': f"{train_recall[i]-test_recall[i]:+.4f}"
        },
        {
            'Class': class_name,
            'Metric': 'F1-Score',
            'Training (10-Fold CV)': f"{train_f1[i]:.4f}",
            'Testing': f"{test_f1[i]:.4f}",
            'Difference': f"{train_f1[i]-test_f1[i]:+.4f}"
        }
    ])

# Add overall metrics
overall_metrics = [
    {
        'Class': 'Overall',
        'Metric': 'Accuracy',
        'Training (10-Fold CV)': f"{train_accuracy:.4f}",
        'Testing': f"{test_accuracy:.4f}",
        'Difference': f"{train_accuracy-test_accuracy:+.4f}"
    },
    {
        'Class': 'Overall',
        'Metric': 'ROC-AUC',
        'Training (10-Fold CV)': f"{train_roc_auc:.4f}",
        'Testing': f"{test_roc_auc:.4f}",
        'Difference': f"{train_roc_auc-test_roc_auc:+.4f}"
    },
    {
        'Class': 'Overall',
        'Metric': 'MCC',
        'Training (10-Fold CV)': f"{train_mcc:.4f}",
        'Testing': f"{test_mcc:.4f}",
        'Difference': f"{train_mcc-test_mcc:+.4f}"
    }
]

performance_df = pd.DataFrame(class_performance_data + overall_metrics)
print(performance_df.to_string(index=False))

# STEP 8: Enhanced Performance Visualization for Both Classes (WITH ROC CURVE)
print("\n" + "="*80)
print("📊 GENERATING ENHANCED PERFORMANCE VISUALIZATIONS")
print("="*80)

# Create comprehensive performance comparison for both classes
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(20, 16))

# 1. Main Performance Comparison (Overall Metrics)
overall_metrics = ['Accuracy', 'ROC-AUC', 'MCC']
train_overall = [train_accuracy, train_roc_auc, train_mcc]
test_overall = [test_accuracy, test_roc_auc, test_mcc]

x = np.arange(len(overall_metrics))
width = 0.35

bars1 = ax1.bar(x - width/2, train_overall, width, label='Training (10-Fold CV)',
               color=elsevier_colors[0], alpha=0.8, edgecolor='black', linewidth=1.5)
bars2 = ax1.bar(x + width/2, test_overall, width, label='Testing (Held-Out)',
               color=elsevier_colors[1], alpha=0.8, edgecolor='black', linewidth=1.5)

# Add value labels on bars
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                f'{height:.3f}', ha='center', va='bottom',
                fontweight='bold', fontsize=11)

ax1.set_xlabel('Performance Metrics', fontsize=14, fontweight='bold')
ax1.set_ylabel('Score', fontsize=14, fontweight='bold')
ax1.set_title('XGBoost (Tuned) - Overall Performance\nTraining vs Testing',
              fontsize=16, fontweight='bold', pad=20)
ax1.set_xticks(x)
ax1.set_xticklabels(overall_metrics)
ax1.legend(fontsize=12, framealpha=0.9)
ax1.set_ylim(0, 1.1)
ax1.grid(True, alpha=0.3, linestyle='--')

# 2. ROC CURVE for Training and Testing Sets
ax2.plot(fpr_train, tpr_train, color=elsevier_colors[0], lw=2.5,
         label=f'Training ROC (AUC = {train_roc_auc:.3f})', alpha=0.8)
ax2.plot(fpr_test, tpr_test, color=elsevier_colors[1], lw=2.5,
         label=f'Testing ROC (AUC = {test_roc_auc:.3f})', alpha=0.8)

# Plot diagonal reference line
ax2.plot([0, 1], [0, 1], color='gray', lw=1.5, linestyle='--', alpha=0.7,
         label='Random Classifier (AUC = 0.5)')

# Fill area under curves
ax2.fill_between(fpr_train, tpr_train, alpha=0.2, color=elsevier_colors[0])
ax2.fill_between(fpr_test, tpr_test, alpha=0.2, color=elsevier_colors[1])

ax2.set_xlabel('False Positive Rate', fontsize=14, fontweight='bold')
ax2.set_ylabel('True Positive Rate', fontsize=14, fontweight='bold')
ax2.set_title('ROC Curves - Training vs Testing Sets\nXGBoost (Tuned)',
              fontsize=16, fontweight='bold', pad=20)
ax2.legend(loc='lower right', fontsize=12, framealpha=0.9)
ax2.grid(True, alpha=0.3, linestyle='--')
ax2.set_xlim([0.0, 1.0])
ax2.set_ylim([0.0, 1.05])

# 3. Confusion Matrix - Training
im1 = ax3.imshow(train_cm, cmap='Blues', interpolation='nearest', alpha=0.8)
ax3.set_xlabel('Predicted Label', fontsize=14, fontweight='bold')
ax3.set_ylabel('True Label', fontsize=14, fontweight='bold')
ax3.set_title('Training Set - Confusion Matrix\n(10-Fold CV Average)',
              fontsize=16, fontweight='bold', pad=20)
ax3.set_xticks([0, 1])
ax3.set_yticks([0, 1])
ax3.set_xticklabels(class_names)
ax3.set_yticklabels(class_names)

# Add text annotations for training CM
for i in range(train_cm.shape[0]):
    for j in range(train_cm.shape[1]):
        ax3.text(j, i, f'{train_cm[i, j]}\n({train_cm[i, j]/np.sum(train_cm):.1%})',
                ha='center', va='center', fontsize=12, fontweight='bold',
                color='white' if train_cm[i, j] > np.max(train_cm)/2 else 'black')

# 4. Confusion Matrix - Testing
im2 = ax4.imshow(test_cm, cmap='Reds', interpolation='nearest', alpha=0.8)
ax4.set_xlabel('Predicted Label', fontsize=14, fontweight='bold')
ax4.set_ylabel('True Label', fontsize=14, fontweight='bold')
ax4.set_title('Testing Set - Confusion Matrix\n(Held-Out Dataset)',
              fontsize=16, fontweight='bold', pad=20)
ax4.set_xticks([0, 1])
ax4.set_yticks([0, 1])
ax4.set_xticklabels(class_names)
ax4.set_yticklabels(class_names)

# Add text annotations for testing CM
for i in range(test_cm.shape[0]):
    for j in range(test_cm.shape[1]):
        ax4.text(j, i, f'{test_cm[i, j]}\n({test_cm[i, j]/np.sum(test_cm):.1%})',
                ha='center', va='center', fontsize=12, fontweight='bold',
                color='white' if test_cm[i, j] > np.max(test_cm)/2 else 'black')

plt.tight_layout()
plt.savefig('xgboost_tuned_comprehensive_performance.png', dpi=300,
            bbox_inches='tight', facecolor='white')
plt.show()

# STEP 9: FIXED Classification Reports
print("\n" + "="*80)
print("📋 DETAILED CLASSIFICATION REPORTS FOR BOTH CLASSES")
print("="*80)

print("\nTRAINING SET CLASSIFICATION REPORT (10-Fold CV Average):")
training_report = classification_report(y_train_encoded, cv_predictions,
                                      target_names=class_names, digits=4)
print(training_report)

print("\nTESTING SET CLASSIFICATION REPORT (Held-Out):")
testing_report = classification_report(y_test_encoded, y_test_pred,
                                     target_names=class_names, digits=4)
print(testing_report)

# STEP 10: Individual Class Performance Analysis
print("\n" + "="*80)
print("🎯 INDIVIDUAL CLASS PERFORMANCE ANALYSIS")
print("="*80)

for i, class_name in enumerate(class_names):
    print(f"\n{class_name.upper()} CLASS PERFORMANCE:")
    print(f"  Training (10-Fold CV):")
    print(f"    Precision: {train_precision[i]:.4f}")
    print(f"    Recall:    {train_recall[i]:.4f}")
    print(f"    F1-Score:  {train_f1[i]:.4f}")
    print(f"  Testing (Held-Out):")
    print(f"    Precision: {test_precision[i]:.4f}")
    print(f"    Recall:    {test_recall[i]:.4f}")
    print(f"    F1-Score:  {test_f1[i]:.4f}")
    print(f"  Performance Gap:")
    print(f"    Precision: {train_precision[i]-test_precision[i]:+.4f}")
    print(f"    Recall:    {train_recall[i]-test_recall[i]:+.4f}")
    print(f"    F1-Score:  {train_f1[i]-test_f1[i]:+.4f}")

# STEP 11: Tuning Results Summary
print("\n" + "="*80)
print("🔬 HYPERPARAMETER TUNING SUMMARY")
print("="*80)

print(f"🎯 BEST PARAMETERS FOUND:")
for param, value in best_params.items():
    print(f"   {param}: {value}")

print(f"\n📊 PERFORMANCE COMPARISON:")
print(f"   Default Model Accuracy: {accuracy_default:.4f}")
print(f"   Tuned Model Accuracy:   {accuracy_tuned:.4f}")
print(f"   Improvement:            {improvement:+.4f}")

print(f"\n🚀 FINAL TUNED MODEL PERFORMANCE:")
print(f"   Training CV Accuracy: {train_accuracy:.4f}")
print(f"   Test Accuracy:        {test_accuracy:.4f}")
print(f"   Test ROC-AUC:         {test_roc_auc:.4f}")
print(f"   Test MCC:             {test_mcc:.4f}")

# STEP 12: Save ROC Data for Later Use (LIKE OTHER MODELS)
print("\n" + "="*80)
print("💾 SAVING ROC DATA FOR COMBINED VISUALIZATIONS")
print("="*80)

# Save the tuned model
joblib.dump(best_xgb_model, 'xgboost_tuned_model.pkl')
joblib.dump(label_encoder, 'xgboost_tuned_label_encoder.pkl')

# ============== SAVE ROC PREDICTION DATA ==============
print("\n💾 SAVING ROC PREDICTION DATA FILES...")

# File 1: Save NPZ file for ROC plotting
np.savez('xgboost_roc_predictions.npz',
         # Training set data (from cross-validation)
         y_train_true=y_train_encoded,
         y_train_prob=cv_probabilities[:, 1],  # Probability for positive class

         # Testing set data
         y_test_true=y_test_encoded,
         y_test_prob=y_test_proba[:, 1],  # Probability for positive class

         # Precomputed ROC curve points
         fpr_train=fpr_train,
         tpr_train=tpr_train,
         fpr_test=fpr_test,
         tpr_test=tpr_test,

         # AUC values
         train_auc=train_roc_auc,
         test_auc=test_roc_auc,

         # Metadata
         model_name='XGBoost',
         class_names=class_names,
         feature_count=len(feature_columns),
         best_params=str(best_params))
print("✅ Saved: xgboost_roc_predictions.npz")

# File 2: Save detailed predictions CSV
predictions_df = pd.DataFrame({
    'true_label': y_test_encoded,
    'predicted_label': y_test_pred,
    'probability_class_0': y_test_proba[:, 0],
    'probability_class_1': y_test_proba[:, 1],
    'correct': (y_test_encoded == y_test_pred)
})
predictions_df.to_csv('xgboost_predictions.csv', index=False)
print("✅ Saved: xgboost_predictions.csv")

# File 3: Save comprehensive metrics CSV
metrics_df = pd.DataFrame({
    'Model': ['XGBoost'],
    'Train_Accuracy': [train_accuracy],
    'Test_Accuracy': [test_accuracy],
    'Train_AUC': [train_roc_auc],
    'Test_AUC': [test_roc_auc],
    'Train_Precision_0': [train_precision[0]],
    'Train_Precision_1': [train_precision[1]],
    'Test_Precision_0': [test_precision[0]],
    'Test_Precision_1': [test_precision[1]],
    'Train_Recall_0': [train_recall[0]],
    'Train_Recall_1': [train_recall[1]],
    'Test_Recall_0': [test_recall[0]],
    'Test_Recall_1': [test_recall[1]],
    'Train_F1_0': [train_f1[0]],
    'Train_F1_1': [train_f1[1]],
    'Test_F1_0': [test_f1[0]],
    'Test_F1_1': [test_f1[1]],
    'Train_MCC': [train_mcc],
    'Test_MCC': [test_mcc],
    'Num_Features': [len(feature_columns)],
    'N_Estimators': [best_params.get('n_estimators', 'N/A')],
    'Max_Depth': [best_params.get('max_depth', 'N/A')],
    'Learning_Rate': [best_params.get('learning_rate', 'N/A')]
})
metrics_df.to_csv('xgboost_metrics.csv', index=False)
print("✅ Saved: xgboost_metrics.csv")

# File 4: Save all predictions (training + testing) for reference
all_predictions_df = pd.DataFrame({
    'dataset': ['train'] * len(y_train_encoded) + ['test'] * len(y_test_encoded),
    'true_label': np.concatenate([y_train_encoded, y_test_encoded]),
    'predicted_label': np.concatenate([cv_predictions, y_test_pred]),
    'probability_class_1': np.concatenate([cv_probabilities[:, 1], y_test_proba[:, 1]])
})
all_predictions_df.to_csv('xgboost_all_predictions.csv', index=False)
print("✅ Saved: xgboost_all_predictions.csv")

# File 5: Save feature importance (XGBoost specific)
feature_importances = best_xgb_model.feature_importances_
importance_df = pd.DataFrame({
    'Feature': feature_columns,
    'Importance': feature_importances
}).sort_values('Importance', ascending=False)
importance_df.to_excel('xgboost_feature_importances.xlsx', index=False)
print("✅ Saved: xgboost_feature_importances.xlsx")

# File 6: Save grid search results
grid_results_df = pd.DataFrame(grid_search.cv_results_)
grid_results_df.to_excel('xgboost_grid_search_results.xlsx', index=False)
print("✅ Saved: xgboost_grid_search_results.xlsx")

# File 7: Save tuning parameters
tuning_params = {
    'best_params': best_params,
    'best_cv_score': grid_search.best_score_,
    'default_accuracy': accuracy_default,
    'tuned_accuracy': accuracy_tuned,
    'improvement': improvement,
    'test_roc_auc': test_roc_auc,
    'test_mcc': test_mcc
}
tuning_params_df = pd.DataFrame([tuning_params])
tuning_params_df.to_excel('xgboost_tuning_parameters.xlsx', index=False)

print(f"\n🎯 XGBOOST WITH HYPERPARAMETER TUNING COMPLETED!")
print(f"📊 Training (10-Fold CV) Accuracy: {train_accuracy:.1%}")
print(f"📊 Testing Accuracy: {test_accuracy:.1%}")
print(f"📈 Training ROC-AUC: {train_roc_auc:.3f}")
print(f"📈 Testing ROC-AUC: {test_roc_auc:.3f}")
print(f"⚡ Improvement over default: {improvement:+.2%}")
print(f"🔍 Performance metrics shown for BOTH classes: {class_names}")

print(f"\n📁 FILES GENERATED FOR ROC CURVES:")
print(f"   1. xgboost_roc_predictions.npz     - Main ROC data file")
print(f"   2. xgboost_predictions.csv         - Detailed test predictions")
print(f"   3. xgboost_metrics.csv             - Performance metrics")
print(f"   4. xgboost_all_predictions.csv     - All predictions (train+test)")
print(f"   5. xgboost_tuned_model.pkl         - Trained model")
print(f"   6. xgboost_feature_importances.xlsx - Feature importance scores")
print(f"   7. xgboost_grid_search_results.xlsx - Grid search results")

print(f"\n🎯 Use 'xgboost_roc_predictions.npz' with other model files for combined ROC plots!")
print("🚀 TUNED XGBOOST MODEL READY FOR DEPLOYMENT!")

### **Naive Bayes:**

Requirements: training-set-selected-features.xlsx, testing-set-selected-features.xlsx

In [ ]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                           roc_auc_score, roc_curve, matthews_corrcoef, confusion_matrix,
                           classification_report)
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score, StratifiedKFold, cross_val_predict, GridSearchCV
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

# Set publication-quality parameters (Fixed font settings)
plt.rcParams['font.size'] = 12
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['axes.titlesize'] = 16
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
plt.rcParams['legend.fontsize'] = 12
plt.rcParams['figure.titlesize'] = 18

# Use available system fonts to avoid warnings
plt.rcParams['font.family'] = ['DejaVu Sans', 'Arial', 'Helvetica', 'sans-serif']

# Professional Elsevier color scheme
elsevier_colors = ['#2E86AB', '#A23B72', '#F18F01', '#C73E1D', '#1A936F', '#114B5F']

print("🚀 NAIVE BAYES CLASSIFIER WITH HYPERPARAMETER TUNING (10-FOLD CV + GRID SEARCH)")
print("="*80)

# STEP 1: Load the feature-selected datasets
print("📊 Loading feature-selected datasets...")

train_df = pd.read_excel('training_set_selected_features.xlsx')
test_df = pd.read_excel('testing_set_selected_features.xlsx')

print(f"Training set: {train_df.shape}")
print(f"Testing set: {test_df.shape}")

# STEP 2: Prepare features and target
print("\n" + "="*80)
print("🔧 PREPARING DATA")
print("="*80)

# Identify feature columns (exclude metadata)
metadata_columns = ['COMPOUND ID', 'SMILE CODE', 'Ligand Type']
feature_columns = [col for col in train_df.columns if col not in metadata_columns]

X_train = train_df[feature_columns]
y_train = train_df['Ligand Type']

X_test = test_df[feature_columns]
y_test = test_df['Ligand Type']

print(f"Selected features: {len(feature_columns)}")
print(f"Training samples: {X_train.shape[0]}")
print(f"Testing samples: {X_test.shape[0]}")

# Encode labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# FIXED: Properly get class names as strings
class_names = [str(cls) for cls in label_encoder.classes_]
print(f"Class names: {class_names}")

print("✅ Data prepared successfully!")

# STEP 3: Hyperparameter Tuning with GridSearchCV and 10-Fold CV
print("\n" + "="*80)
print("🎯 HYPERPARAMETER TUNING WITH GRIDSEARCHCV (10-FOLD CV)")
print("="*80)

# Define parameter grid for Naive Bayes
param_grid = {
    'var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1]
}

# Initialize Naive Bayes
nb = GaussianNB()

# 10-fold cross-validation strategy
cv_strategy = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# GridSearchCV with 10-fold CV
print("🔍 Performing Grid Search with 10-fold CV...")
print(f"   Parameter grid: {param_grid}")
print(f"   Total parameter combinations: {len(param_grid['var_smoothing'])}")
print(f"   This may take a while...")

grid_search = GridSearchCV(
    estimator=nb,
    param_grid=param_grid,
    cv=cv_strategy,
    scoring='accuracy',
    n_jobs=-1,
    verbose=1
)

# Perform grid search
grid_search.fit(X_train, y_train_encoded)

print("✅ Grid Search completed!")

# Display best parameters and scores
print(f"\n🎯 BEST PARAMETERS FOUND:")
best_params = grid_search.best_params_
for param, value in best_params.items():
    print(f"   {param}: {value}")
print(f"   Best CV Score: {grid_search.best_score_:.4f}")

# STEP 4: Compare Default vs Tuned Models
print("\n" + "="*80)
print("📊 COMPARISON: DEFAULT vs TUNED PARAMETERS")
print("="*80)

# Train model with default parameters for comparison
nb_default = GaussianNB()
nb_default.fit(X_train, y_train_encoded)
y_pred_default = nb_default.predict(X_test)
accuracy_default = accuracy_score(y_test_encoded, y_pred_default)

# Use best model from grid search
best_nb_model = grid_search.best_estimator_
y_pred_tuned = best_nb_model.predict(X_test)
accuracy_tuned = accuracy_score(y_test_encoded, y_pred_tuned)

improvement = accuracy_tuned - accuracy_default

print(f"   Default Parameters Accuracy: {accuracy_default:.4f}")
print(f"   Tuned Parameters Accuracy:   {accuracy_tuned:.4f}")
print(f"   Improvement:                 {improvement:+.4f}")

if improvement > 0:
    print(f"   ✅ Tuning improved accuracy by {improvement:.2%}")
else:
    print(f"   ⚠️  Tuning did not improve accuracy")

# STEP 5: 10-Fold Cross-Validation with Tuned Model (for proper comparison)
print("\n" + "="*80)
print("🎯 10-FOLD CROSS-VALIDATION WITH TUNED PARAMETERS")
print("="*80)

print("🚀 Performing 10-fold cross-validation with tuned parameters...")

# Get cross-validated predictions for training set using tuned parameters
cv_predictions = cross_val_predict(best_nb_model, X_train, y_train_encoded,
                                 cv=cv_strategy, method='predict')
cv_probabilities = cross_val_predict(best_nb_model, X_train, y_train_encoded,
                                   cv=cv_strategy, method='predict_proba')

# Calculate training metrics as average across 10-fold CV
train_accuracy = accuracy_score(y_train_encoded, cv_predictions)

# Calculate metrics for BOTH CLASSES
train_precision = precision_score(y_train_encoded, cv_predictions, average=None)
train_recall = recall_score(y_train_encoded, cv_predictions, average=None)
train_f1 = f1_score(y_train_encoded, cv_predictions, average=None)
train_roc_auc = roc_auc_score(y_train_encoded, cv_probabilities[:, 1])
train_mcc = matthews_corrcoef(y_train_encoded, cv_predictions)

# Also get individual fold accuracies for reporting
cv_scores = cross_val_score(best_nb_model, X_train, y_train_encoded,
                          cv=cv_strategy, scoring='accuracy', n_jobs=-1)

print("✅ 10-fold cross-validation completed!")

print(f"\n📊 10-FOLD CROSS-VALIDATION RESULTS (TRAINING PERFORMANCE):")
print(f"   Fold scores: {[f'{score:.4f}' for score in cv_scores]}")
print(f"   Mean CV Accuracy: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")
print(f"   Standard Deviation: {cv_scores.std():.4f}")

# STEP 6: Make predictions on TEST set with tuned model
print("\n" + "="*80)
print("📊 MODEL EVALUATION WITH TUNED PARAMETERS")
print("="*80)

# Predictions on TEST set with tuned model
y_test_pred = best_nb_model.predict(X_test)
y_test_proba = best_nb_model.predict_proba(X_test)

# Calculate metrics for TESTING set for BOTH CLASSES
test_accuracy = accuracy_score(y_test_encoded, y_test_pred)
test_precision = precision_score(y_test_encoded, y_test_pred, average=None)
test_recall = recall_score(y_test_encoded, y_test_pred, average=None)
test_f1 = f1_score(y_test_encoded, y_test_pred, average=None)
test_roc_auc = roc_auc_score(y_test_encoded, y_test_proba[:, 1])
test_mcc = matthews_corrcoef(y_test_encoded, y_test_pred)

# Confusion matrices
train_cm = confusion_matrix(y_train_encoded, cv_predictions)
test_cm = confusion_matrix(y_test_encoded, y_test_pred)
tn, fp, fn, tp = test_cm.ravel()

# Additional metrics
specificity = tn / (tn + fp)
npv = tn / (tn + fn)

# Performance gap
accuracy_gap = train_accuracy - test_accuracy

# STEP 7: Display comprehensive results for BOTH CLASSES
print("\n" + "="*80)
print("📈 COMPREHENSIVE PERFORMANCE METRICS FOR BOTH CLASSES (TUNED MODEL)")
print("="*80)

# Create detailed comparison dataframe for BOTH CLASSES
class_performance_data = []

for i, class_name in enumerate(class_names):
    class_performance_data.extend([
        {
            'Class': class_name,
            'Metric': 'Precision',
            'Training (10-Fold CV)': f"{train_precision[i]:.4f}",
            'Testing': f"{test_precision[i]:.4f}",
            'Difference': f"{train_precision[i]-test_precision[i]:+.4f}"
        },
        {
            'Class': class_name,
            'Metric': 'Recall',
            'Training (10-Fold CV)': f"{train_recall[i]:.4f}",
            'Testing': f"{test_recall[i]:.4f}",
            'Difference': f"{train_recall[i]-test_recall[i]:+.4f}"
        },
        {
            'Class': class_name,
            'Metric': 'F1-Score',
            'Training (10-Fold CV)': f"{train_f1[i]:.4f}",
            'Testing': f"{test_f1[i]:.4f}",
            'Difference': f"{train_f1[i]-test_f1[i]:+.4f}"
        }
    ])

# Add overall metrics
overall_metrics = [
    {
        'Class': 'Overall',
        'Metric': 'Accuracy',
        'Training (10-Fold CV)': f"{train_accuracy:.4f}",
        'Testing': f"{test_accuracy:.4f}",
        'Difference': f"{train_accuracy-test_accuracy:+.4f}"
    },
    {
        'Class': 'Overall',
        'Metric': 'ROC-AUC',
        'Training (10-Fold CV)': f"{train_roc_auc:.4f}",
        'Testing': f"{test_roc_auc:.4f}",
        'Difference': f"{train_roc_auc-test_roc_auc:+.4f}"
    },
    {
        'Class': 'Overall',
        'Metric': 'MCC',
        'Training (10-Fold CV)': f"{train_mcc:.4f}",
        'Testing': f"{test_mcc:.4f}",
        'Difference': f"{train_mcc-test_mcc:+.4f}"
    }
]

performance_df = pd.DataFrame(class_performance_data + overall_metrics)
print(performance_df.to_string(index=False))

# STEP 8: Enhanced Performance Visualization (WITH ROC CURVE)
print("\n" + "="*80)
print("📊 GENERATING ENHANCED PERFORMANCE VISUALIZATIONS")
print("="*80)

# Compute ROC curves for training and testing
fpr_train, tpr_train, _ = roc_curve(y_train_encoded, cv_probabilities[:, 1])
fpr_test, tpr_test, _ = roc_curve(y_test_encoded, y_test_proba[:, 1])

# Create comprehensive performance comparison
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(20, 16))

# 1. Main Performance Comparison (Overall Metrics)
overall_metrics = ['Accuracy', 'ROC-AUC', 'MCC']
train_overall = [train_accuracy, train_roc_auc, train_mcc]
test_overall = [test_accuracy, test_roc_auc, test_mcc]

x = np.arange(len(overall_metrics))
width = 0.35

bars1 = ax1.bar(x - width/2, train_overall, width, label='Training (10-Fold CV)',
               color=elsevier_colors[0], alpha=0.8, edgecolor='black', linewidth=1.5)
bars2 = ax1.bar(x + width/2, test_overall, width, label='Testing (Held-Out)',
               color=elsevier_colors[1], alpha=0.8, edgecolor='black', linewidth=1.5)

# Add value labels on bars
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                f'{height:.3f}', ha='center', va='bottom',
                fontweight='bold', fontsize=11)

ax1.set_xlabel('Performance Metrics', fontsize=14, fontweight='bold')
ax1.set_ylabel('Score', fontsize=14, fontweight='bold')
ax1.set_title('Naive Bayes (Tuned) - Overall Performance\nTraining vs Testing',
              fontsize=16, fontweight='bold', pad=20)
ax1.set_xticks(x)
ax1.set_xticklabels(overall_metrics)
ax1.legend(fontsize=12, framealpha=0.9)
ax1.set_ylim(0, 1.1)
ax1.grid(True, alpha=0.3, linestyle='--')

# 2. ROC CURVE for Training and Testing Sets
ax2.plot(fpr_train, tpr_train, color=elsevier_colors[0], lw=2.5,
         label=f'Training ROC (AUC = {train_roc_auc:.3f})', alpha=0.8)
ax2.plot(fpr_test, tpr_test, color=elsevier_colors[1], lw=2.5,
         label=f'Testing ROC (AUC = {test_roc_auc:.3f})', alpha=0.8)

# Plot diagonal reference line
ax2.plot([0, 1], [0, 1], color='gray', lw=1.5, linestyle='--', alpha=0.7,
         label='Random Classifier (AUC = 0.5)')

# Fill area under curves
ax2.fill_between(fpr_train, tpr_train, alpha=0.2, color=elsevier_colors[0])
ax2.fill_between(fpr_test, tpr_test, alpha=0.2, color=elsevier_colors[1])

ax2.set_xlabel('False Positive Rate', fontsize=14, fontweight='bold')
ax2.set_ylabel('True Positive Rate', fontsize=14, fontweight='bold')
ax2.set_title('ROC Curves - Training vs Testing Sets\nNaive Bayes (Tuned)',
              fontsize=16, fontweight='bold', pad=20)
ax2.legend(loc='lower right', fontsize=12, framealpha=0.9)
ax2.grid(True, alpha=0.3, linestyle='--')
ax2.set_xlim([0.0, 1.0])
ax2.set_ylim([0.0, 1.05])

# 3. Confusion Matrix - Training
im1 = ax3.imshow(train_cm, cmap='Blues', interpolation='nearest', alpha=0.8)
ax3.set_xlabel('Predicted Label', fontsize=14, fontweight='bold')
ax3.set_ylabel('True Label', fontsize=14, fontweight='bold')
ax3.set_title('Training Set - Confusion Matrix\n(10-Fold CV Average)',
              fontsize=16, fontweight='bold', pad=20)
ax3.set_xticks([0, 1])
ax3.set_yticks([0, 1])
ax3.set_xticklabels(class_names)
ax3.set_yticklabels(class_names)

# Add text annotations for training CM
for i in range(train_cm.shape[0]):
    for j in range(train_cm.shape[1]):
        ax3.text(j, i, f'{train_cm[i, j]}\n({train_cm[i, j]/np.sum(train_cm):.1%})',
                ha='center', va='center', fontsize=12, fontweight='bold',
                color='white' if train_cm[i, j] > np.max(train_cm)/2 else 'black')

# 4. Confusion Matrix - Testing
im2 = ax4.imshow(test_cm, cmap='Reds', interpolation='nearest', alpha=0.8)
ax4.set_xlabel('Predicted Label', fontsize=14, fontweight='bold')
ax4.set_ylabel('True Label', fontsize=14, fontweight='bold')
ax4.set_title('Testing Set - Confusion Matrix\n(Held-Out Dataset)',
              fontsize=16, fontweight='bold', pad=20)
ax4.set_xticks([0, 1])
ax4.set_yticks([0, 1])
ax4.set_xticklabels(class_names)
ax4.set_yticklabels(class_names)

# Add text annotations for testing CM
for i in range(test_cm.shape[0]):
    for j in range(test_cm.shape[1]):
        ax4.text(j, i, f'{test_cm[i, j]}\n({test_cm[i, j]/np.sum(test_cm):.1%})',
                ha='center', va='center', fontsize=12, fontweight='bold',
                color='white' if test_cm[i, j] > np.max(test_cm)/2 else 'black')

plt.tight_layout()
plt.savefig('naive_bayes_tuned_comprehensive_performance.png', dpi=300,
            bbox_inches='tight', facecolor='white')
plt.show()

# STEP 9: Classification Reports
print("\n" + "="*80)
print("📋 DETAILED CLASSIFICATION REPORTS FOR BOTH CLASSES")
print("="*80)

print("\nTRAINING SET CLASSIFICATION REPORT (10-Fold CV Average):")
training_report = classification_report(y_train_encoded, cv_predictions,
                                      target_names=class_names, digits=4)
print(training_report)

print("\nTESTING SET CLASSIFICATION REPORT (Held-Out):")
testing_report = classification_report(y_test_encoded, y_test_pred,
                                     target_names=class_names, digits=4)
print(testing_report)

# STEP 10: Individual Class Performance Analysis
print("\n" + "="*80)
print("🎯 INDIVIDUAL CLASS PERFORMANCE ANALYSIS")
print("="*80)

for i, class_name in enumerate(class_names):
    print(f"\n{class_name.upper()} CLASS PERFORMANCE:")
    print(f"  Training (10-Fold CV):")
    print(f"    Precision: {train_precision[i]:.4f}")
    print(f"    Recall:    {train_recall[i]:.4f}")
    print(f"    F1-Score:  {train_f1[i]:.4f}")
    print(f"  Testing (Held-Out):")
    print(f"    Precision: {test_precision[i]:.4f}")
    print(f"    Recall:    {test_recall[i]:.4f}")
    print(f"    F1-Score:  {test_f1[i]:.4f}")
    print(f"  Performance Gap:")
    print(f"    Precision: {train_precision[i]-test_precision[i]:+.4f}")
    print(f"    Recall:    {train_recall[i]-test_recall[i]:+.4f}")
    print(f"    F1-Score:  {train_f1[i]-test_f1[i]:+.4f}")

# STEP 11: Tuning Results Summary
print("\n" + "="*80)
print("🔬 HYPERPARAMETER TUNING SUMMARY")
print("="*80)

print(f"🎯 BEST PARAMETERS FOUND:")
for param, value in best_params.items():
    print(f"   {param}: {value}")

print(f"\n📊 PERFORMANCE COMPARISON:")
print(f"   Default Model Accuracy: {accuracy_default:.4f}")
print(f"   Tuned Model Accuracy:   {accuracy_tuned:.4f}")
print(f"   Improvement:            {improvement:+.4f}")

print(f"\n🚀 FINAL TUNED MODEL PERFORMANCE:")
print(f"   Training CV Accuracy: {train_accuracy:.4f}")
print(f"   Test Accuracy:        {test_accuracy:.4f}")
print(f"   Test ROC-AUC:         {test_roc_auc:.4f}")
print(f"   Test MCC:             {test_mcc:.4f}")

# STEP 12: SAVE ALL ROC DATA FOR LATER USE
print("\n" + "="*80)
print("💾 SAVING ROC DATA FOR COMBINED VISUALIZATIONS")
print("="*80)

# Save the tuned model
joblib.dump(best_nb_model, 'naive_bayes_tuned_model.pkl')
joblib.dump(label_encoder, 'naive_bayes_tuned_label_encoder.pkl')

# ============== SAVE ROC PREDICTION DATA ==============
print("\n💾 SAVING ROC PREDICTION DATA FILES...")

# File 1: Save NPZ file for ROC plotting
np.savez('naive_bayes_roc_predictions.npz',
         # Training set data (from cross-validation)
         y_train_true=y_train_encoded,
         y_train_prob=cv_probabilities[:, 1],  # Probability for positive class

         # Testing set data
         y_test_true=y_test_encoded,
         y_test_prob=y_test_proba[:, 1],  # Probability for positive class

         # Precomputed ROC curve points
         fpr_train=fpr_train,
         tpr_train=tpr_train,
         fpr_test=fpr_test,
         tpr_test=tpr_test,

         # AUC values
         train_auc=train_roc_auc,
         test_auc=test_roc_auc,

         # Metadata
         model_name='Naive Bayes',
         class_names=class_names,
         feature_count=len(feature_columns))
print("✅ Saved: naive_bayes_roc_predictions.npz")

# File 2: Save detailed predictions CSV
predictions_df = pd.DataFrame({
    'true_label': y_test_encoded,
    'predicted_label': y_test_pred,
    'probability_class_0': y_test_proba[:, 0],
    'probability_class_1': y_test_proba[:, 1],
    'correct': (y_test_encoded == y_test_pred)
})
predictions_df.to_csv('naive_bayes_predictions.csv', index=False)
print("✅ Saved: naive_bayes_predictions.csv")

# File 3: Save comprehensive metrics CSV
metrics_df = pd.DataFrame({
    'Model': ['Naive Bayes'],
    'Train_Accuracy': [train_accuracy],
    'Test_Accuracy': [test_accuracy],
    'Train_AUC': [train_roc_auc],
    'Test_AUC': [test_roc_auc],
    'Train_Precision_0': [train_precision[0]],
    'Train_Precision_1': [train_precision[1]],
    'Test_Precision_0': [test_precision[0]],
    'Test_Precision_1': [test_precision[1]],
    'Train_Recall_0': [train_recall[0]],
    'Train_Recall_1': [train_recall[1]],
    'Test_Recall_0': [test_recall[0]],
    'Test_Recall_1': [test_recall[1]],
    'Train_F1_0': [train_f1[0]],
    'Train_F1_1': [train_f1[1]],
    'Test_F1_0': [test_f1[0]],
    'Test_F1_1': [test_f1[1]],
    'Train_MCC': [train_mcc],
    'Test_MCC': [test_mcc],
    'Num_Features': [len(feature_columns)]
})
metrics_df.to_csv('naive_bayes_metrics.csv', index=False)
print("✅ Saved: naive_bayes_metrics.csv")

# File 4: Save all predictions (training + testing) for reference
all_predictions_df = pd.DataFrame({
    'dataset': ['train'] * len(y_train_encoded) + ['test'] * len(y_test_encoded),
    'true_label': np.concatenate([y_train_encoded, y_test_encoded]),
    'predicted_label': np.concatenate([cv_predictions, y_test_pred]),
    'probability_class_1': np.concatenate([cv_probabilities[:, 1], y_test_proba[:, 1]])
})
all_predictions_df.to_csv('naive_bayes_all_predictions.csv', index=False)
print("✅ Saved: naive_bayes_all_predictions.csv")

# File 5: Save grid search results
grid_results_df = pd.DataFrame(grid_search.cv_results_)
grid_results_df.to_excel('naive_bayes_grid_search_results.xlsx', index=False)
print("✅ Saved: naive_bayes_grid_search_results.xlsx")

print(f"\n🎯 NAIVE BAYES WITH HYPERPARAMETER TUNING COMPLETED!")
print(f"📊 Training (10-Fold CV) Accuracy: {train_accuracy:.1%}")
print(f"📊 Testing Accuracy: {test_accuracy:.1%}")
print(f"📈 Training ROC-AUC: {train_roc_auc:.3f}")
print(f"📈 Testing ROC-AUC: {test_roc_auc:.3f}")
print(f"⚡ Improvement over default: {improvement:+.2%}")
print(f"🔍 Performance metrics shown for BOTH classes: {class_names}")

print(f"\n📁 FILES GENERATED FOR ROC CURVES:")
print(f"   1. naive_bayes_roc_predictions.npz     - Main ROC data file")
print(f"   2. naive_bayes_predictions.csv         - Detailed test predictions")
print(f"   3. naive_bayes_metrics.csv             - Performance metrics")
print(f"   4. naive_bayes_all_predictions.csv     - All predictions (train+test)")
print(f"   5. naive_bayes_tuned_model.pkl         - Trained model")
print(f"   6. naive_bayes_grid_search_results.xlsx - Grid search results")

print(f"\n🎯 Use 'naive_bayes_roc_predictions.npz' for creating combined ROC plots!")
print("🚀 TUNED MODEL READY FOR DEPLOYMENT!")